In [8]:
from fastapi import FastAPI, HTTPException
from helper_functions import get_llm, extract_transcript, load_embeddings, store_embeddings

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import json 
from pydantic import BaseModel
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
# from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import time
from fastembed.embedding import DefaultEmbedding
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import time
# from qdrant_client import QdrantClient
from qdrant_client.http import models

# from transformers import pipeline
# from langchain.embeddings import HuggingFaceInstructEmbeddings
import pickle
# import faiss
# from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
documents = """anyways [Music] thank you [Music] hey everyone we have an exciting live stream today we are here with Daniel as our guest who's going to be teaching us storybook I keep hearing so much about it so I can't wait to see how it works I actually saw uh Daniel do a talk in Portugal on it and it was super interesting before we get into that Daniel please give yourself an intro and a shout out okay cool hey uh that's nice to be here thanks for having me on uh my name is Danny uh I'm also a British guy but I live in I live in Porto I work for a company called cover Flex We Do flexible benefits for like uh for employees and yeah basically I'm the the way I'm involved in storybook is that I maintain a couple of uh packages for react native and um yeah I uh just really came on storybook I like the community around it and hopefully I can show you a bit about it and um also storybook is a open source project so we know this is a hectober first thing and basically you know if you really like storybook you can also get involved on the storybook repository I love that so yeah you being a core maintainer you're encouraging people to contribute I think that's just awesome thank you so much Danny just to give a shout out to some people in the chat as well uh akshaya you were first welcome and we've got Emma Sashi and Carl and Kota prolu hopefully I pronounce the right awesome to have you all here we're actually going to be using the Eddie Hub link free project uh next JS Branch as an example here so we can see um if it's something we want to continue using or we can learn more about it and see if it's something that um uh it will be useful to us so uh yeah Danny I can say Daniel so Danny has has prepared a great live stream for us today so I'm super excited so what is Storybook I suppose yeah so storybook um is like a it's a toolkit for building UI and um essentially it gives you um some tools of course to to to build UI in a sort of isolated sandbox environment so you're able to like if you uh if you can see you're seeing my screen right now yeah that's correct yes cool so basically if you look at what's on the screen right now this is actually on the storybook website and uh nice enough to have a little GIF here with the it shows a little bit of an example here so essentially you can see um some like individual components on the screen and you can expand that into more complex components and or even you know go down to the lower level and each of these little things on the side here these are called what we call stories um or in fact this thing's here with the little book my bookmark icon and we'll see a bit more of that uh further in I'm sure and essentially you can do a couple of things here you can like change some of the props so if you're familiar with react you know you you know that when you have a component usually you have a couple of props that you want to change and you're able to do that without having to you know go into the code you can just uh they take a little toggle and I think they even have a little in this demo here they do a little thing where they change the time yeah he's doing it now and yeah basically you can use this in your development process um to basically simplify things and uh for example I know that in this project the link free project you're using a Docker image and you know uh that's pretty nice uh and it's it's like an easy way to get a environment set up but even simpler way to go is nothing right so if you just run storybook you don't even need the the database or anything running you're just using the components without the whole you know page around it and everything that sounds really interesting okay I'm super super excited now awesome and uh I think it'll really helping boarding onboarding people to the project they ought to see like we're not trying to go through the code see what components there are this will list what components are available to them if they want to create a new page and they want to use different components they can use existing components and just check what props are available um yeah exactly that's also like another one of those use cases for it which I didn't really cover just then but yeah you can do documentation as well in your storybook and some of the stuff that you know uh you saw in my talk from devtools Porto was about the you know the features like um automatic accessibility testing and like Integrations with figma and basically there's a load of different add-ons that extend the functionality of storybook to really make it that toolkit that you know you can use to help you build up components and we're you know um even things like mocking API endpoints so um I think we probably won't get too far into that today in terms of the API endpoints but that's something that you can consider as well and uh what the way you usually do it with storybook is you set up a mock service worker which is like a cool little package uh you know you can get from npm and it just lets you mock your API endpoints and if you have all your mocks set up you can even just plug that straight into storybook and it will work and and that's another thing you know like if you have that setup for storybook or you just have it set up for that so you can run like a mock version of your app they can share that config and uh it's in a little uh it doesn't require a lot more work to get that set up basically nice I think that sounds super interesting um people are already discussing in the chat about having yeah a sub domain where um uh we could um uh yeah have it deploy to and so forth so I think that's really good yeah awesome yeah and I think you can see a couple examples of that I I don't know if if you remember uh well I guess the chat won't know but there's a this if I just bring up this one example uh app which is uh basically uh this is just like a fake app for um showing off storybook and this is built by uh Geico Jan Braga and he's like one of the core Starbucks maintainers and so like you can imagine this is an example app that's based on like a finer restaurant sort of thing like a Okay Google Maps but just for restaurants or something you know and you can like order on here so what they what he's got as well is similar to that it's a It's like a if you go to I think slash storybook here I think it you probably would want it to be a subdomain right so that it doesn't really get that accidentally but then you can see here that app itself has its own Storybook in each one of these things you know like for example this is the footer that's from that app and then if you have that for your link free you know you could go oh wait how did they build that little link thing there and you go to the the storybook real quick and just see uh that uh component and you can even have a look in the docs and see like a it's not happening here but sometimes you could see like the source code oh here it is there's a show code if that'll work yeah you can kind of see something there but yeah there's some cool features like that cool oh I'm super excited so let's uh I know you've got some exciting plans for our link for you today so let's let's see I just realized the script was really small that's okay it looks good um I see the chat game quite quite excited as well we actually have a question from um from Carl uh is it using webpack or Veet for bundling yeah it's actually an interesting question uh so in the in the past uh it's been sort of a webpack only thing but um it doesn't stop you from using it with other bundlers like for example your app doesn't need to be using webpack for you to use storybook and storybook can be used like with things like Ruby on Rails but actually nowadays you can even run storybook with Veet so vit is that bundler that sort of came along with the uh View and the guys that built view right so and that you can use to get really fast uh hot module reloading in um or fast refresh I think they're calling it now with the uh with storybook and there's still some caveats to that where the currently storybook the the apps of around your components will be bundled with webpack but the preview where your components go you get to choose what bundles that so if you want to use V for that you can use it via like uh called the Builder V Builder okay makes sense Let's uh um that sounds good it causes us huge okay perfect yeah and actually you mentioned about slots I just saw his comment he mentioned about swell and yeah spell is like uh getting some nicer support nowadays I think as well I think we had some back and forth with some guys from the swell community and yeah the the support I think a lot of things are really going forward in a good direction for version seven as well so like version seven is coming up pretty soon and there's going to be some big improvements in a lot of areas including performances I think a lot of people worry about um especially when you get really big story books but you know there's already some tools you can use to get around it but in the newer version the the storybook UI is going to be pre-bundled uh but we might be getting a little bit ahead of ourselves yeah maybe it looks like Carl's saying so you've got another project called finder um and we use salt kit for that so they can see uh Carl's gonna test storybooks with uh with the finder project um which will probably be done in about 10 minutes because Carl is super fast he says I want to think about doing something before you know it it's already kind of done yeah it was just typing that out he was already in the code base yeah also so should we should we dive into the the project let's do it and I'm also thinking about you mentioned accessibility and I know Emma um is is a fan of accessibility so I can't wait to improve our link free project that's getting a lot of traction as well now awesome yeah so let's have a look first I guess we have this next.js Branch right this is yes but you've been building out like a is a new version as far as I understand right of the link free uh app and yeah so basically what we're gonna do is get on this branch and uh do the basic setup uh in the next JS project there's like one or two like extra steps that we will have to do but it's actually pretty nice because there's an add-on that sort of handles the complicated stuff so we don't actually have to do anything really complicated ourselves and um yeah me personally I never really worked much with next.js even Nexus with storybook but uh you know just looking around and figuring out it seems it's actually pretty simple and I didn't have any issues uh figuring out so I think uh nice if we go as we go through it here you'll see there's nothing super complicated here a lot of the steps it's just knowing okay these are the commands that we need to run um and actually have a well you know what let's I'll bring up the projects here so hopefully this is big enough everyone can see uh there's obviously not much to see right now but if I bring up the bring up some file in here that looks familiar yeah yeah oh let me just delete that we don't want anything sort of residual here yeah let's start from scratch so we can all learn yeah exactly I think that should be a clean State here and I'm on that so if anyone wanted to follow along I suppose they could as well and um so essentially what we want to do is uh run a little init command from storybook so uh storybook has this nice little SD CLI which of course sends this storybook and here if you run the init command um that will basically just set up everything uh the basics for you for storybook and at that point you can kind of already just run uh the storybook and it will have some sort of basic uh component setup from you know like example components they're not like from your app in that case and you have to add those yourself that would be really good that's in the next that's in the version AIDS yeah maybe we'll see how they how they feel about that awesome so yeah let's just run that and see hopefully it doesn't take too long um but it shouldn't do and you uh for anyone who's wondering what MPX is it's just like uh for instead of having to install this CLI you can just run commands from that package and it's a built-in feature of npm all right yeah thanks for clarifying that I make so many assumptions and so that's good that we shouldn't make assumptions so thank you yeah no worries so yeah now it's just going to go through and install a few things and if you look over here you can already see it's added back that folder that I deleted which you didn't no one saw that uh and so we've got a couple of files in here already well so whilst that's installing we could even just sort of Let It hide away for a bit and we'll have a look at like what files we have here if I just want to format that yeah so we have this main.js and that is funny enough the main config file and what goes in there is like some of the details about how your storybook is set up and you know getting started probably won't need to adjust much in here and we won't actually really touch that and if you saw in that command that I ran I had a little flag to say use webpack five I got you um that's I don't think that's super necessary but it's nice to use the newer versions and in Starbucks 7 they're actually deprecating webpack for but either way if you don't know what webpack is don't even worry about it [Music] yeah and if you wanted to use like the question earlier if you wanted to use you essentially just put like the hit okay you would have to install it as well so there'll be a package I think in this case we'll be able to see it uh I don't know maybe it's not finished it's not finished installing them yet but we can we could check that later maybe I think it actually just finished as I clicked away which is always uh always nice uh yeah so here you go I think yeah so we've got like build a webpack five and so you would in this case you would uh install V so this is not super important if you're if you're not sure about webpack and V but if you did want to know how to get the web Builder uh it's easier probably to just search on Google beat Builder storybook and you'll probably find everything you need to know but the basics is you just Swap this out and install the package so yeah other than that we have this stories um array here and this is basically like saying like where can I find the stories and the default setup when you run the command it's going to put them in a stories folder and it will look for anything any subfolders in there with and any files you know in the nested structure that have stories uh dot you know any one of these extensions so jsjsx TSX TS I think and this project is just JS so technically you probably only need those but um it doesn't hurt to happen more but yeah so basically this will tell storybook where the stories are and this is like a glob syntax so um I wouldn't worry too much about the specifics of it but it's kind of like a version of regex and cervical use that to find your files um so and then we have these add-ons here these are the ones that come by default when you when you set up a fresh project and Essentials here has a bunch of like really cool stuff like the the controls thing which is what lets you adjust props on the Fly um and a few other things but we uh we'll get into that I suppose when we actually run it um and just one more sort of thing here the preview we can set up a couple other things here and they're sort of like runtime config whereas these are more like uh bundle like uh build Time stuff so and and these preview things you can put here like jsx and things like that um so even here you can have like a decorators which are like wrappers for your component but um I think we can skip past that for now because it's not super important we won't need to use any of those things there's only one thing we'll have to add to this preview file and we'll get to that in a moment so like I said earlier as well we have this stories folder and they set up some example uh components and some example stories so if we just take a look here the there's for example like a page with a perhaps we look at a simpler on a button sure so that we have a button component um and this is like uh just a regular react component nothing about Storybook in here this is your if you had like a component in here you know this you could replace this with whatever other component and what you do in your story file which is what Storybook is looking for is you import that uh you know that component wherever it may be and then just tell storybook this story is about that component and this default uh export is what storybook's looking for for a lot of like metadata and things to just basically tell it what to do and a lot most of these are sort of optional things to be honest and the simplest story you just don't need any of this basically and it will give you uh a title and you know the args that's only if you want to use do the props on the fly so I'll if the simplest story you know if I just took like got rid of most of this it's just that basically and so the story itself is this here this export primary and that's essentially this template buying thing don't worry about it it's basically just exporting this template thing here which is a function that uh is returning that component and so essentially it's just a component itself um and so your story is like a component with a couple of extra things and then storybook picks it up and you know knows what to do with it so maybe a lot of this is probably not making a lot of sense right now but if we run the storybook yeah running I want to see what we've got we can yeah I just I was giving it time to finish running oh okay no worries I'm getting excited I'm like seeing this code I want to see how it looks no no you're doing the right thing so I get too excited no worries no worries no but but you're right to want to see it I think we should go straight to it if possible so I think there's a little thing that happens here because the s-link file isn't uh in the format it was expecting that we can just sort of um put it put this in there so that this is an optional step but storybook has a yes plugin so you could actually when you're writing your stories get suggestions about you know like oh you shouldn't do that because whatever I I don't know if I'll look in a specific example right now but I'm sure it will probably tell me something I'm doing is wrong at some point so now this has run all the way through it gives you a little output here saying yeah to run your storybook type um storybook and that's yarn is not required so don't worry about that we can just run uh with the npm syntax so yarn is like a just a a replacement or an alternative to npm so in case anyone's not familiar that's why it's saying to do yarn storybook it's the same as npm room storybook can I jump in with a question yeah you've got a question from uh Carl they say they found a blog post and Storybook of swelt but it uses version 7 Alpha would you recommend to use Alpha already or or not I think that really depends on you know what your you know your situation the project that you're working on like I I know a lot of like uh sort of more corporate setups you know where you you don't want to use anything that is sort of not uh stable and but like smaller startups or just personal projects it doesn't really matter as much you know what I mean and I think in the in the case of storybook which is more like a Dev tool it's not super important if it's not like a stable version you know what I mean like uh if if your storybook breaks it's annoying but it doesn't break for your users which at the end of the day is like the important thing you know so Carl I recommend if you found something good on the newer version which is it's gonna do you know when the newer version is coming out uh Danny it's roughly uh uh I'm not sure and I wouldn't want to give the wrong answer and that's the kind of uh thing to ask uh probably some of the the more of the web team uh no problem no I can't think of it like top of my head but sure no problem I think but it's gonna be released I think the the thing is the great thing is that these these releases are coming out so Carl I'd say go with the alpha um it doesn't break the app as also as Danny mentioned so um yeah so that's good yeah so and that's the thing like they are Super Active on developing that so you'll see like the releases like every every other day sometimes you know and they're just pushing out uh the new stuff and so they have full-time maintainers working on storybook uh storybook is sort of uh has a a company that funds a few people to be like core like maintainers full time so uh it's definitely it just keeps moving along so uh you don't have to worry about like an alpha being stuck in Alpha forever um that's good yeah are you able to make that a bit bigger it's a sort of full screen as well please okay it's okay is that is that good that's good yeah okay so now looking at this um one thing you can maybe do I don't know if this is a terrible idea but let me just see because you know this is sort of the thing that you might do when you're developing on Starbucks or what you can do if you have two monitors it's easier right yeah but I guess in all development you've kind of got that thing I do I put my thing over there on the left screen and then put my thing over here so just to sort of show like what happens what's what over here and over there you know so like I was saying um this is a story right so if I just put back whatever was here yeah okay I can sort of just explain what each thing is I don't know if I missed something all right here we go there we go yeah change the title that's why it was confused so okay okay so basically yeah I've got this button and this uh is a component just a regular rack button and I've told storybook that is the component and then I've said okay my story what I want my story to be is just you know render this component with the arguments that I passed through it and those arguments uh per story you can Define them and so in this case we've said okay for the primary one um it's uh let's set primary to true and so you can see when I clicked on this story um that's what's happened here it's set this to true so now if you see when it's false it's a white button and when it's true it's a it's a blue button and so this is like some of those like uh uh basics of storybook where you kind of just set up a state of your component and like a like a slice or how would I put a yeah I don't know like a I'm forgetting the word now but like I sort of a state or an example um someone help us out in the chat let us know um but kind of like uh okay we'll come back to that we'll come back we've kind of got a view of that component in that situation a possible state of this component uh like Frozen in time you can imagine but then at the same time we've got this control Z on which is one of the things that gets included by default and that lets you kind of take uh those args uh or which are you know in sober gland we refer to the what you might say are props in react we say uh args in storybook and so one of those arcs here is the primary Boolean Fields right and storybook has picked up the types of each argument and just said okay yeah this is a Boolean and this is a string and it'll do its best to figure out basically and that's why we've got a couple of things in here that we haven't even sort of listed yet but they're coming from you know uh when you pass this button saying that this is the component server goes and figures out like what are the arguments that I can pass to that and then it will sort of render those out to you in the controls panel and now I can sort of go in here and just say oh what does this button look like if I you know wrote like a whole paragraph in there maybe puts a new line caught you yeah and now you can kind of see like oh I'm just maybe I'm trying to like test all the edge cases for a component I can do that really quickly here because I don't need to go and like set up some mock data or something I just go ahead and type it in the browser nice by the way um Emma says it's a variants the different variants of the button yes exactly I would never have got that I don't know why that word would never have come to mind it's like so yeah and there's even some cool things here like we have this that's so cool background color changer thing uh Color Picker and um this comes up here well actually it Starbucks is able to guess even sometimes when it's a color prop so like if you give it a certain name so if we go back to the the preview that I sort of jumped past before if you look if the names it has this mattress thing which has some regex so if the name is background or color it will assume that the argument type is a color and then it'll give you the Picker but if it's not you can just force it to be that so that's why here I think because it's background color and not background they've just gone they've specified that is the con the color type so then so you if you even if it doesn't pick up the types that your props have you can sort of force it but yeah I mean and in the simplest version of storybook you can you can work completely without the controls even so if I just close that out and you just create these sort of Frozen uh States or these variants of your components uh set up by just like saying you know these are the args the default ones that I want to show when this renders so like for example this secondary one is like the same as the primary one except primary is false and so it's white you know in this case I need to reset the the the state of this so that it goes back to normal but yeah and then in oh whoops what have I done put that down close it yeah there we go sorry but yeah so in a simple storybook you could just set up these variants of your components and just flip through them and then something you can do is like you know maybe you have some automation to do some screenshots of each story and that's something that there are products that do that for storybook for example so you can have some automatic visual testing of like different states of your app or different states of your components um and yeah so like and you can see also here like some documentation um this is MDX built in you know that runs in storybook and MDX for those that don't know is like the markdown format if you're used to GitHub a lot of the stuff you type in to GitHub you can write uh in the markdown format and markdown that has an extension uh well there's an extension of markdown called MDX which is like jsx inside of markdown and there's a lot of like acronyms here but it's just basically saying you can put interactive stuff inside of documentation and you can put that into storybook so um and that's what this docs tab is all about as well so storybook could even generate some default uh documentation for you so I just full screen this I don't think there's there's nothing set up you know I haven't written any markdown for this button component but it's gone ahead and said okay we know this about this component and um they'll just render out in this nice little format and you can see like the different rarence here and you can see the source code I think I'm not sure I've not used this feature the the source code thing too much but I think there's a way to get to show a bit more information as well but yeah the the what's nice about it is like if you just wanted to come and see what the props are you just come here to the docks and see okay there's a this is a thing a primary is of type Boolean and the default is false I think that's great I think it just makes it like documentation takes time but if you have to have this documentation generated and then you're getting extra benefits people can actually interact with the component as well and I see you've got Pages there as well so it's just you're investing um in something that um and you're importing the component so it's more likely to be kept up to date as well I think like one thing that I think um it depends on your posture as well like if you're adding components to storybooks after the fact it can feel a bit like extra work but if you're working on your components from the beginning inside of storybook it just feels natural you know what I mean it's like I'm if I say like you were building link free from the beginning and you know you wanted to add a link component and you have a design for the component and you just come in here and you know you build up the component and then once you're done it looks at the way you want it to you can bring it over to your app or you know even have a story for that page um and then you know creating the story for it was part of the development process and it helped to you know debug things and Etc and and then you know it's not like that extra work you have to do it's just uh you know a tool that helps you whilst you're developing so so we have like tdd right so this would be like sdds a story driven development yeah actually sort of a thing that oh really I was joking yeah okay that's close I think so chromatic uh uh I guess it makes that bigger by the way yeah sorry so there's this article from gramatic which is component driven development and you know I think it's just that thing where like they had to do it of course there to do it because it's like um you know you've got yeah we've got test driven development and then yeah we do kind of have like a this idea of component driven development and I think it does make a lot of sense because when you work with react or you know a lot of uh Frameworks today that is kind of what you're doing naturally and when you put it into storybook it it's almost like it's not just components like um so I don't know if if you know about like the atom molecule what's that thing called with the I can't remember what they call it but the the storybook idea was sort of built up of this um this modular framework for building UI which is is sort of like where you have um you know your building bricks uh that are made up of like uh the smallest possible thing is an atom and then the next is a molecule I can't remember exactly you know what they what's that called but uh the idea is like imagine you know component development that's what you do right and react but if you take it to the next level and in storybook you'll take the component completely out of the app and just on its own in isolation you're truly just developing one component at a time and there's nothing sort of getting in the way or like I don't have to navigate through my app to get there right and I think that's like what they're getting at with like component driven development you know Ms saving the day again Atomic design everything so much this is right it's always good to have people in the chat right exactly I can't remember anything you were hit we hit I always say we're here to start the conversation and then other people can then jump in and um and add even more to the conversation uh before we get to the next bit um Carl asks does it add quite a few dependencies would you say it bloats the size of your project production blow you know it doesn't go to production these are all development dependencies and I don't think uh you know I don't think it's huge but it's no by no means like a little you know two kilobyte package and it also depends on how many add-ons you install like all the extra features that you're getting out of it so you know if if you know your project's already huge and maybe there's a trade-off there you might consider that but I think to me you know it brings enough value to say that it's worth it and none of that is going to my end users at the end of the day because you know I don't deploy the storybook as an app it's like I'm using it during development I can deploy it as a way of like showing off the designers or QA to get like their feedback on things but like I said before with the if it's down you know it's only for us internally and it's not uh so so like yeah there are quite a few dependencies and but like I think the trade-off is there like if it gives you value makes your development process faster let's say you know maybe add a minute to your install time but saves you you know a minute every day for the rest of you know your development time or something yeah so you have to kind of put it that way but yeah I would say it's not small but it's also not massive it's somewhere there in the middle sounds good I think it's you're trying to say that it adds value so it doesn't it's although it adds adds size but it adds value and it's not being deployed to production which I think is uh also important yeah and I think you just have to be a little bit careful that you don't allow those things it shouldn't by default nothing should go into your production build but just don't do something silly like import help the storybook package into your like component itself or something you know and then you should be fine and by default all of the documentation and all the like init commands and stuff will always install things as Dev dependencies in your project so you shouldn't get anything as like you know proper like production dependencies that sounds good right well okay so any more questions or do we should we no let's go let's go I'm super excited so yeah I mean I think we sort of come with a lot of the stuff that's in here but we could maybe also just fill it fiddle around with a few things here just to show you know like some of the other features that you get so out of the box right so we've got this little measurement um like inspect add-on that you can use without having to like open up the development tools and you can see you know this button is you know 44 by 14 and I think even if I bring up the accessibility thing uh maybe it's not got it here but anyway yeah you've got a couple of other tools you can even like zoom in here if you if you can't quite see the the difference in the colors or if there's like a one pixel thing you know that's always useful for like UI you can like refresh the state of the component so like if you're in a more complex component you might want to go back to like a normal State I don't know if there's one in here that no probably not but yeah um if I just disable that I think there's also yeah here we go so we've got things like you know adjusting the size if I just bring the zoom back to normal yeah we can adjust it to different viewports but we can also Define custom viewports and there's a viewports like default set of viewports that you can get from Starbucks as well you just have to add a small add-on that has like a list of like really good ones so like some like things like iPhone and stuff like that that you can use so all this stuff is like yeah I could open up my development tools and you know go through that but then if you look at sort of what happens in and you do that is you've got to sort of go through all of this uh where am I and you know in the storybook UI I feel like it's it's really set up in a really nice way so you can just quickly you know play around with it and you don't have to be developer to know how to use it which I think is really cool so like designers can can pick up on it as well yeah definitely bridges that Gap and you mentioned accessibility which I think again is is super awesome hey pretty much awesome to have you here um actually ask the question is it like WordPress no so this doesn't this is not building your app this is um helping the development of your app so it's so we can have a look at what components are available which we're going to get into now so you're joining it at a good time yeah so you know it's exactly what you say that you don't build in like the UI of storybook is just to let you visualize the stuff that you're building in codes it's not a no code tool you are coding but it will just add some like stuff in there for you for free and you know um I think if you feel like WordPress if you want something more like that it's storybook's not that kind of tool basically um but yeah let's get back into this I think I've basically shown there's a bunch of these little add-ons that come in here by for free and you can sort of see uh you know what each uh just little details about your component and sort of debug a few things like this one could be useful I don't know if you've ever done this when you're building an app but you're trying to figure out where that outlines of a component are and you just put down like borders around every single little component maybe you can kind of avoid that by using some of these features you know that sounds good yeah and just one more thing I suppose I could just show before we get into some setting up the actual project uh We've also got actions which are basically just um events you can track the events that are getting called when you click on things or type or whatever and so in this case I can see that when I clicked that I got an on click event and see some information about it and I think you can see more information for like more Complex events so that can be useful in some cases and the interactions thing probably will leave for another time but that's also a real feature that lets you do testing inside of storybook and in a visual way um so if you if we have time maybe at the end we can even do like a quick example and it might be pretty cool I think so let's actually set this up for link free and so right now what we've got of running these example components and these are like all the ones I have here these aren't just like um coming from nowhere these are so if I just deleted all of these then I won't have any components anymore and it will complain at me probably that's why just delete all of that because we don't really need that anymore uh so now we've got an empty stories folder if I go back here yeah it says your storybook is empty and it'll tell you that probably no story files or you know you didn't Define the right regex so um one more thing that I want to do before we do that so just so that we can get next yes working nicely um with the project if so if you remember we kind of uh run through this with this init command I update the this link and I'm basically just gonna um now install a little add-on for for next.js and what this does is just kind of there are some like Niche things in in next.js that are like really useful in when you're developing your app but in storybook uh it's a bit more like complicated so for example like next image component has like some special features for like caching and stuff and you know like server-side things and that's not really going to work in our setup where we're just sort of rendering react and and so this will just sort of solve that for us without having to um do any extra config so essentially all I'm going to do is run npm install and I'm going to save it to Dev my Dev dependencies so that it doesn't go into my production bundle um and yeah we just give that a second to install and whilst it's doing that we can do the other step we need to do once we've installed which is just add that to our add-ons list if we just come in here what all I've done is I've added this uh the name of that add-on and that will basically just tell storybook okay I need to call some code in in this package um and that's it essentially all of the config that you would have to do manually probably some webpack magic you'd have to do because it's sort of handled by this component and this is also something that's coming in version seven where next year's support would be like first class support so you won't probably even need this step uh in the future okay sounds good and is that in version seven on uh Alpha already do you know I'm not totally sure but no worries I can look at that later you should always give it a try and see what happens yeah the the main thing that you'll notice if you're not using it is basically the image component so if you're not using the next image component you might not even notice anything necessarily um but yeah we'll just see you'd have to try it out but for now I think it's safe to use this uh it's just like a default thing and it's it's a really it works really well like from from my experience let me see I think I had a never mind we'll go back to the hopefully it's gonna finish this installing soon so we can there we go nice just as I start complaining uh okay so one more thing that we're going to want to do since this project is a Tailwind uh it's set up with Tailwind we just want to make sure that those Styles get imported into store book or they get applied so one thing that I mentioned earlier is that the preview.js is sort of like a runtime uh config in a way or just like extras that you can add to the to your like previews um so that's why it's called preview because you know you have that component that renders in the middle that's what we call the preview in store book and if you want to apply anything to that part of storybook that's where you want to put you you're basically always going to the preview and in this case all we need to do is basically import these style files so if you look in This Global CSS you can see here there's some like Tailwind stuff and all this stuff we want in the storybook as well because otherwise we won't get those nice uh styles from the classes that you use in Tailwind right so if I just go and go to this preview.js and import that Styles files being the wrong thing I guess there we go so yeah I basically just importing those Styles and now type type Tailwind so it should work uh in um in the storybook so but we won't be able to say anything yet because we like we like I mentioned earlier we don't have any stories anymore um but so let's go ahead and fix that I guess um what I'm going to do in this case is add a folder to stories called components and in there I'll make a new file and before I do that I'll just mention what I'm what we're going to look at first uh there's a component here which is nice and simple which is why I'm sort of going with this one and it's just a little uh it's a little button component okay so I don't know let me see yeah so that's the we'll see it in a second when I bring it up but um we're just gonna make a story for this component and it does have some like requests in here but it's not super important if it fails so we shouldn't get any issues just rendering this without mocking things and this component takes in a couple of uh parameters so awesome props rather and so we can even you know make use of the args uh that storybook has and the controls there so we can you know adjust the username just by like typing in the UI so you know without further Ado I'll just add a file in the stories uh in the folder that I just created in stories I'm going to call it use the link dot stories oh JS and some of you might already be thinking well you know why don't you make it sort of alongside the component and that's actually something you can do so you know if I wanted to I can just go and update here so it'll also look in the components folder for stories um but in Nexus there's a this whole concept of like file based routing and so when we want to make a story for a page in our we can't just put it directly in the pages folder because it's going to assume that's a page of our app and so we'd end up having to make a separate folder for our stories anyway so in this case I've gone ahead and just said okay well let's have a a folder you know let's put all of our stories in the stories folder um but if you're not using xjs or you're not using file based routing routing you could put them alongside like I could just make a file in here called with the same name sorry I I do I do like it how it's in a separate folder though like some people put their automated testing in the same folder as the source code and I actually do like it like it's separate um maybe following a similar sort of structure um but have it have but sometimes it doesn't always make sense it depends on the project but having it separate I I prefer this way so this I think looks uh looks good I don't know what everyone else thinks in the chat let us know but um I do like the idea of having it in a separate folder yeah and think like it almost done to preference right and it shouldn't make any difference at all really other than just you know look and feel or you know Dev experience so if it feels better to you to have it alongside the component you can do that and you know you have that uh option in storybook so you're not limited to the stories folder um I'm just putting it there because in this case I think it makes a lot more sense for a nexjs project and so yeah we've got like stories file but we're missing a few things here so since this is like sort of the Crux of things let's let's do a bit of sort of live recording so I'm gonna go ahead and import that component so if we look here we're in stories components we've got to go back a couple of folders so I've gone ahead and uh you know that's why I've got these two dots and yeah I'm importing from components users user link and that's that component I was just mentioning which is a button right and so as I mentioned earlier what we want to do is basically tell a story with what our component is and we do that in the default export we don't need the title for now but what we can do is just say yeah the component is user link and you know that's why essentially the reason why we've been we've imported it there why is it I should okay so this is I said that the ear sling thing is going to come in handy somewhere and this is basically what it will do so you know you've got a story file you don't add any stories to it so it's not going to show anything and like yeah that's a good point actually I should add a story so that's what I'm going to do next so in in storybook the stories are just the exports and the default like the the named exports here so if I say like const uh yeah default sure copilot is definitely helpful yeah I know exactly and um so essentially this would be a named export and this is a default export right and the default export is like the information for storybook and the the default I mean the named export is the one um that is the actual story and it's kind of confusing because I've named this one default I'm gonna I'm gonna call it primary I don't know like uh I like to do basic I don't know uh but you can kind of call it whatever you want I'm terrible with naming this is where the community comes in and really helps but what we can always rename it later so basically is yeah and and you know that's the thing in programming naming is the most difficult problem of all so so yeah basically what we can do is this is like the basic um as we've named it basic as well it's the basic story it's just a it's a component that's taking arcs from storybook and passing them to uh the component itself which is the user link and then here what I can do is say yeah like it's profile is suggesting I can go ahead and say like oh here's the args for basic and what that means is like the default props that I want to pass to it so that when I click there you know that's the variant that we're you know trying to display and I have sort of set this up in advance and figure out what the user link is sort of looking for and so I know already what those are but and whilst you're developing is so you would probably uh sort of fig add them in as you go I think so here what we're looking for is a is a link and that has a URL so it can be Google and it has some clicks and uh also a name so we can say in this case I think what we're looking at are links you know on the link free page or links to content about you right so this would be like my YouTube channel or something right exactly and so this this doesn't make as much sense now if I just put youtube.com there you go and uh not sponsored by YouTube by the way but yeah so just username there's also another one that is expecting and then this uh displays that's public so if we just go and look here does exactly what I was talking about so link username and DisplayPort stats public um now this should be all we need to get this rendering in our storybook and what's going to happen is we'll see um this user link um will be the uh the thing that shows up in the sidebar and then it will have a little story called basic and instead of just talking about it let's actually run this and see our new story with userlink and we should see a little a button with my YouTube channel written there so we'll just give that a second to build looking forward to it so I just think it's so useful for people um working on the project or even new to the project to visually see um like kind of what the options are without having to dig into the into the code trying to think how people do it now and look at documentation that has been written might be our date so this is uh this is really good yeah and seeing from my own experience I had to sort of dig out what those props meant you know exactly what's a link you know what is it expecting inside the link you know because the other way that you would look for what these are is kind of go into if I remember correctly it's like a date not the data model link right yes yeah and so I had to sort of dig through that and if we've got these stories set up and you know whilst we're developing you have it in mind what you're expecting from that endpoint you can just set up the story and then the next person that comes along to try and you know add to your uh component knows okay well it's got these compo these things already I just need to you know maybe you want to add like uh I don't know whatever here something like a description I don't know and then you just all you have to do is sort of figure out whether the name's coming from and and then you can add a description and but you don't need to figure out much else uh but yeah I think it's exactly like that you know the you can also like I said see the components that already exist and maybe reuse some stuff uh and instead of like creating two and some later on someone goes why haven't made that component yes which does happen yeah exactly so yeah and you can do a lot of custom stuff as well like you could have like one page at the front with like a bunch of your components just showing off on the on the documentation page so here we go enough stalling here we go we have a link being rendered so that's you can as you can see that's what I wrote in the story itself my YouTube channel if I want I can just come in here and say you know uh twitch you know or something and you already you can see an update and in this case here we have like a sort of more nested structure for this prop and that's why we've got this sort of Json editor coming up here and it's pretty nice actually that it's kind of picked that up and said oh here you go you can sort of edit that in a nice way but um I guess these ones aren't actually visualized in the component they're more useful like when you click it it's gonna like try and make a request right so yeah but yeah I mean that is essentially like you're set up now you've got that story in there and now everything else that you want to do you can do from there and let's you know if you were developing this and let's say you know you wanted to I don't know maybe change the color or something now you've you know you've you're in a situation where you could easily you can see that component in isolation as well I think yeah exactly you could easily see okay well um I don't really know any of these things but let's get rid of one of these or something and I can say how does it look if I get rid of that got you no and I don't have to you know navigate through an app or run the docker uh you know I'm just here already like a new user I wouldn't have to go through the steps that I went through today you know like I've installed storybook and everything they just come in you know run npm install and then run npm run storybook and then they can you know already get going and updating some components no it makes complete sense I'm really really liking this um especially when you have different versions variants I should say of of the component I think is awesome yeah exactly and I I think it's actually um I think we discussed this a little bit before but I think it's a really cool way of getting people uh you know getting new contributions as well to a project so doing your first contribution so like your first contribution could just be adding a a new story for one of the components exactly and you can get your hacktoberfest going basically I'm liking that oh yeah this could be an idea for some uh some tweets later in the week so perfect nice right so um what I was thinking next is we could do a sort of more complicated um component and in this case an actual like a full page so um it's unfortunate I don't have the maybe if I can just go to link free like I'll go to your link for you maybe and show you like what link3 actually looks like because I don't know if I'm I suppose people will know right most most should uh the the downside is the the version of xjs branch is not deployed um so uh but it's okay you don't have to show it but they'll see it within um within here right yeah I'll just do it real quick just so that we know like what we're expecting to see a rough idea is not exactly like this everyone because I haven't done icons yet there's no colors yet so um the new version going from react to next JS does have a is not as feature-rich at the moment but it does have other features like statistics and clicks and stuff like that but that would be perfect for some some contributions right exactly yeah so if you see stuff that's missing here you can go in and add it right so but yeah and I think like so what we're going to kind of try and do is get you know like a base this basic page with the features that are in the next JS version showing in storybook and in a way that's just kind of like a mock version of the app running locally in a sense this app is a bit you know it's kind of like a a one-page app in some ways right because you've just got that I don't know that well I don't know if there's some hidden stuff in there but um we can kind of show the whole app in storybook even just by putting in the I think what's the page called uh let me see it's the username page I guess um so if I go back to my vs code um and we'll maybe I'll even zoom in a bit more just so you can see what I'm looking at in the sidebar here so if we look through this project we've got the pages folder so and this is that thing we were talking about earlier is the file based routing and the username file is um this is a page depending on which user is and it's going to show their their link free page right yeah this is the one that we're gonna try and make a story for okay cool and this one is kind of interesting actually because it does do some uh server request stuff but it's doing it on the get server side props function so and for people who don't know about Nexus that's just like do this on the server but our actual component is here and then it will get the props from the server side from this function right and I'm not super familiar with Nexus but I think that's how it works so this data will be coming in in the real app will be coming from get server size props right yes so basically you don't need to do the skeleton loading like you're doing react because the component won't be displayed until it's already got the data um so we actually had someone add to the branch skirt and loading and then put a sleep of like 500 milliseconds to try and so you could see otherwise you would never see a skirts unloading and I was like no we don't we don't need it we want to get away from that whole skeleton loading stuff that's okay and so but the reason I bring it up is because that's actually kind of useful for us right now because every thing every bit of information is actually in this data object so what that means is if we want to render that in storybook all we have to do is take this user uh component okay and pass it the the props that would be would be you know present in the database essentially correct would be the response from get server side props so I don't even need to Mark anything in this case I just you know put some args there or the one ARG which would be data right um and that's pretty handy and if we go through here there's you can see already there's a couple of things here like name Avatar um you know the links which you know and this is interesting because what we've just done is looked at the user link right and now what we're going to do is we're going to sort of compose a bunch of components together in one component and we kind of you're seeing the building box that you can build up like the atomic design that Emma so nicely reminded us of and and this is like this is that uh you know the big the the final product in a sense and if we had each of the components used here in their own component you could see like the whole all the building blocks coming together in a sense and and in this case what we're going to do is see the whole thing as a whole and to do that well what we're going to do is add another file and another folder even to our stories folder so I'm gonna go back one but you know what it's fine let that be that big let's go okay let's make pages so what I'm trying to do here is kind of follow the structure of the the app so in the stories folder if I wanted to find something I would and let's say Okay I want to find this component I just follow the same structure in here so perhaps I should have made a user folder now I think about it but you get the idea yeah we can we could even do that real quick I'll just put user and then put that in there it might even update the inputs for me it did it did okay nice so that should continue working so that was the idea anyways like you just sort of copy the structure that you have throughout the project and you'll be able to easily find stuff when you're looking for it and you know that's the there's a trade-off here like when you're if you put it like alongside your components it's really easy to find them when you're working on it yeah you have to do a little bit more work but anyway we'll get back to the storybook stuff and in this case we've got pages and then just straight up just straight the the folder there and so we can we can give it the same name as it has oh you can give it the exact same name interesting yeah I mean in in these cases like uh you know your founding can basically have anything in right and sorry book's not stopping you from doing anything as long as you give it you know in your main.js just configure it to say like you know this is what I'm looking for sure we've got to hear this star dot uh stories and so it could be literally anything there as long as it's got stories on the end and uh one of these extensions so yeah we can now go ahead and just for example import that user component which I think is the default export me you know I think too many things open now so I'll just close everything else and bring up username so if we look here that is the default export so I can just straight up import it like this and be nice if here we go yeah pages and then username I think you need to go up oh yeah you're right yeah yeah three things yeah you're right I even have it next to me and I'm still doing it wrong no worries it happens so what is it complaining I think it's because I haven't used it yeah there you go so yeah that's the right path now we just do the same as we did in the other one I'm gonna give it a default export with some information I'm gonna put the user in there which is the component and just close that off and that that again is just the the basic stuff we can we can give it a title and even just say this is you know and say that it's the the user component but it does that by default but what you can do here is kind of like um you could even give it some sort of more complicated structure and say like uh but what they've done recently is made it so that it will automatically check your file structure and then put it under like Pages user so I'm not sure if we'll see that here but I think it should do that once we have more than one component we'll be able to see that a bit better and so now like it was complaining there you know we have to have at least one story and so what we're going to do is create again our basic doesn't it can be called anything but you know as I normally like to create basic just for whatever reason and so we're gonna do the same thing again which is just pass the args to that component and if you don't do that it will also work it just uh won't work with the controls and actually with the newer version of storybook um you don't need any that at all you can just pass an empty object and it already work um this is uh what is coming as part of csf3 which is this this syntax we're writing here is what they called CSF and we can actually use it already it's kind of in a preview state it already works in the current version of storybook so you know I think in this case let's do this one with the with that syntax maybe or would that be to would that be like mixing up too many things yeah yeah keep it simple keep it simple it's I've only got a small brain I can only cope with so much okay so we're going with like the standard uh like this F2 way of doing things and but you'll see in the future as 7.0 comes out for anyone who's interested anyway then there will be changes um and it will be going towards this new format with csf3 where it's actually really simplifies the uh the editing experience of of stories where you half of these things you don't even need to write anymore and like you'll see that this is a kind of like a repeated pattern that we do a lot of the time with the args passed to the component and in the newer version you don't need this at all it just does that by default assumes that's what you want to do and then if you want to do something different you can tell it but anyway yeah we'll go with the basic stuff and so here you know we passed the args and in this case there's a lot more that it's expecting so I'm just going to copy from an example I have um let me just get that I think this should be it so in this case we have a few more details we want to pass yeah we've got the links and we've got the the data and everything and but yeah this is essentially everything that we had to do we just set up that named export and then we set the args for it and these are all the default values like we were like I mentioned before and because it's in our stories folder and it's got the stories extension and those are the things that we Define in our main.js like where we're looking for stories it should now appear if we're lucky it should now appear in here oh something's happening hopefully I haven't done something wrong if nothing goes wrong it's not a live stream so it's it's good never be questioning if it's live or not so one thing you can sometimes do just to make sure is just sort of shouldn't have to do this but sometimes it will catch errors if you just sort of restart the the server the Starbucks server um but generally you know it has hot module refreshing and everything so um if we just give it a second here we might find that there's something sort of error here that where I've put the import wrong or something yeah it might need one more level up I think I think you might need that but I'm not sure I'm just gonna answer um some questions I'm just going to say while you're debugging it yeah so we're possibly going to add this to link free um so therefore there could be some uh maybe the next JS Branch the version two of Link three um so that would be good to be thinking of adding storybook to that this is like an example that Danny's taking us through with link free still but if we go ahead and do it then we're going to need to add all the existing components into the storybook so yeah any help on that appreciated and it will count towards Octoberfest as well okay awesome yeah so I figured out what the problem is if you see here I've put a little comment here saying replace me so that's basically why I forgot to do even though I've reminded myself in the guide um but basically in this uh uh page there's a picture of the user and so it's expecting to receive an image there so what I'm going to do is essentially just add a an image there or I can give you or I can give you a URL to an image as well I I have a I think I have one sort of set up here somewhere okay just it shouldn't take so just take a second here if I just find I'll just save it again but yeah I uh I knew this is gonna happen that's why I wrote that there no problem uh so Vijay says storybook really takes a commit author to the next level yes so um hopefully it uh is gonna help our project but we're interested to know what people's thoughts are we'll continue the conversation in Discord I'll do some screen shares in Discord next week as well um oh so this is the little placeholder image I found online okay uh this should probably be something you know uh be something else because I don't know if this is like a public domain or not but just for this demonstration sure yeah or use that and this is actually a PNG in this case so it should now stop complaining um so if I go back refresh oh it's because the Rand failed the Run failed here so we just run that again we sort of you know pray to the coding Gods we should get you should get this working um yeah this question about Oktoberfest but I do so much content Twitter space live streams on it and if we get time we'll chat about it in a moment but not just yet I want to focus on link free project and um and storybook and this is how you can get involved in uh hack type of Fest as well ah we have a Pages section yes exactly so that's the thing I was saying Starbucks now is sort of checking your file structure and it will organize the stuff for you but if you want to create your own organization you can also do that and if you just add slashes in the in the title field okay on that metadata it it assumes that is the structure that will take that as the structure and create folders out of that essentially so in this case we've got that uh user thing let me just awesome there we go so this is like you know the most basic version of the link free uh sort of that single link free page with this data and now if you wanted to see you know uh the specific Fields here and like just put like I don't know exclamation mark here you can see already what it's going to look like with different links in there basically no that's great and the um and if people I know someone's added like dark and light mode to not to the the next JS version to the react version but um they'll be able to yeah see that so yeah I think there's just you know different as people add features to components they can see they can kind of see it so I think that's awesome yeah and if you if you kind of sort of if you go ahead with storybook and you want to you know make use of uh those themes and like preview the themes in your storybook there's I think there's something called like Dean's picker or something like that or you can set up a pretty simple way of just swapping themes if you that mail drop example that I showed earlier has one that was set up by Yan um if I just zoom in on this I think there's something in here where you can oh good yeah so in here he's added a this thing here and this is a theme picker basically and he's even added a little thing where you can put them side by side that's very cool so yeah like little things like that if you have a dark mode or all that kind of stuff you can try and you can visualize that and imagine if you're developing um you know maybe let's say an app you would go and like oh I have to change the dark mode each time I want to like change uh you know the styling or I want to add a new component that works for both modes if you have this feature for example you can see both at the same time pretty easily so it's a nice way of like facilitating that that looks really good I like it um I really think I'm looking forward to chatting with the Eddie Hub community in Discord and seeing um I definitely think we should uh do especially the next JS branch on link free um do some more on this um because I think it just adds yeah so much benefit to onboard people I have to look at the mocking and so forth because when you click these buttons it's going to create an API request um to to do the stats I'll have to have a look at how to do that but it sounds interesting yeah I mean if you like I've done that kind of stuff with mocking before so if you need any sort of guidance on that I can help you out but essentially um I think I have a couple of links as well that I added to the that little notes uh thing that I sent you earlier but essentially um you want to use I'll bring it up on screen here there's this add-on called MSW uh storybook add-on okay and in here you can basically forget to an example here you can just say like um if I zoom in a bit more so let's say we had like a a get request to user and you can just sort of say okay when this request goes off just you know intercept that and send this response instead and so yeah this is pretty nice all you have to do really is set up uh install like MSW and install um the the add-on for Starbucks that's uh that looks really really cool um yeah it looks great I think we could definitely do that is that in the documentation on on storybook yes so they have a bit a little section um I'll just bring it up as well now that we're yeah they have a [Music] um a page on the on the official documentation here if you come to this if I zoom in again um we come through this um storybook documentation page so it's on the storybook.js docs you can also if you can go to it pretty easily from the from here you just come to the Box okay uh I guess one of these things guys I think it's guides yeah so in here there's a little section about writing stories and they have a section on building pages and screens and so like in general like for just basic components you won't gen normally won't need that kind of mocking stuff but when it comes to like building Pages there's a couple more things that you might want to consider right and and in this case they sort of list out some of the stuff that you might want to think about so um if we go down here a bit you can see like mocking connected components and mocking API Services Etc and then here they basically suggest that MSW add-on and then they've got some of those examples here as well um but yeah like I think the documentation um you know storybook is really in depth you just have to sort of pick it out of the uh in those pages but anything you need I'm I'm willing to help out and thank you so much maybe I can make a piano for you yeah I was gonna say maybe with the stuff you've done already maybe just uh submit that as a PR now um and uh that can count towards your Oktoberfest nice yeah I actually recently did sign up so far it's just been my own PRS but this would be good to get like a a real one kind of uh in the bag uh so let's see um to do that let's go ahead and make a new Branch I guess yeah and then you probably need to add it as a remote as well yeah yeah but I have one already I think oh okay well normally what I do actually is I use the GitHub CLI and I'll just do like create PR or something oh I haven't used the GitHub CLI so um yeah uh that's pretty cool yeah I'll I'll work in a demo right here okay perfect so um just just to check through the things that I've done here I don't need to check that so I think that's good to go so make sure we don't put any sort of rubbish in in the project I don't know it's kind of working with limited space yeah so yeah so I'll just go through and check real quick I think it seems okay got Starbucks stuff it's great advice to check to check your changes before committing that's good you never want to push a console log like I think probably sixty percent of my PR's go to my colleague and get a comment immediately you left a console lock yeah or when people do uh as yours is just joking it was just uh you know git add dot yeah never use dot um you know probably for the sake of like brevity I should probably just do that right now but in general okay I can get rid of this yarn lock yeah yeah um this seems I think everything's okay so what we'll do is just to get but and I'll do a New Branch I'll call it uh let's see I've got a feature storybook or something yeah and then do the do the merge do the pull request into next JS branch yeah yeah I will do okay so I'll just make a new branch and I'll do mats with these changes so let's do add so set up storybook and I think that should be all we need to do then like you said it'll have to put it on my Fork of the project so what I normally do for that is just creates uh I think that should be enough yeah so I don't want to push it to my branch on my Fork rather uh it looks good this is pretty cool yeah I need to just check that I'm I have some like weird issue with my gut where it doesn't always authenticate so I'll just authenticate off from the side okay and with the template you can just choose a blank one it's fine yeah okay blank one yeah it's fine okay we'll create an issue and I'll update the description so don't don't worry too much about it yeah so I'll just uh what I like to do usually is continue in the browser and then what it does is it opens it up um and now you can see I should probably not open another browser but here we go uh here we go so it's saying right now oh my God it's so zoomed in I can't really zoom out slightly okay so I just need to change this now to next.js oh yes yeah and it should now yeah now it can merge oh my God why are you doing this Okay cool so I ended up using the template anyway but you can just leave it you can just leave it it doesn't matter it's not fixing an issue like [Music] uh and and storybook to the next stress branch that doesn't really make any sense to be honest but it's a different branch you could list the the two components that you did yeah you could say um uh link storybook and or link was it user link user link that's the one and uh and you did the page as well oh uh user profile page yeah cool so I think that's kind of all we need if we go down here we can even see I haven't made any stupid mistakes and the only thing I'm not sure about is probably you want to put like a different image there oh yeah the picture will probably want to yeah delete that so but that's fine we can we can we can do that later cool so now we just this is almost like a demonstration how to make a progress as well but don't probably don't delete everything in the template if you're doing it yourself yes don't do that this is just for the live stream and and to the people out there I don't know if you ever come across this but oh I love it when people allow edits by maintainers because if you don't it small little changes are really like annoying to get through yeah and if someone is not available you can't make changes to their PR to just like get it over the line but enable this that's what I want to say I run into that a lot but yeah so we'll create this pull request and there we go let's see one thing you could do um is if you wanted to you could set up chromatic and then you'd have a storybook Builds on every pull request depending on how you set it up and then you can open that storybook live uh from here you can add a link to the storybook with those changes from the pr and then you can check someone's changes to the component's life and and something I was going to mention it because they actually have a open source uh like pricing thing where you can and you can get it for free if you're on a source project okay so it's worth looking into so yeah look free for open source oh I shouldn't if I click that then it brings up a chat but so if if that's something you want to go down that road I highly recommend applying for the open source um license or whatever it's called and yeah this is really cool because you can do all those free you can get the visual testing for free you can get the the hosting for free and like you can you know that act you get an action for GitHub that will build the storybook for each PR and that will sort of just take your do all your CI stuff for you and you skip all the the setup okay and so we will have a look into I suppose later on when we get a yeah you know a bit further down the line um I'm sure we could do something with GitHub actions like a basic version now just to yeah for sure and there there's one thing called uh test Runner will run through stories and like do quick checks on them so that's also an option but yeah if you you could you could make a build of each storybook and just see that it doesn't fail probably be the simplest thing yeah which is which is which is important I think it's uh yeah good to check that so um this looks good so off the live stream or probably tomorrow I will um get this merged delete the image to have a bit of a play and uh we'll work on this in in Discord I mean this is uh pretty pretty awesome I really like it while we're just doing a closing note because you switch back to the tab that actually has the the storybook because people are joining now Carl's back hey Tom hey shivangi so yeah leave that so this is what the version of Link free version 2 looks like at the moment it's got the number two in the links that's how many times it's been clicked so yes it doesn't look as nice as version one yet but I was going more for functionality and we're moving to Tailwind so actually having to to build this ourselves but using storybook will be great for us to reuse components um and know what availability there is within the project so um big shout out to Danny for taking time you know hour and a half to the minutes of perfect timing um taking us through uh storybook I I really really like it I really can't wait to start using it and then look forward to geeking out everyone next week um on this with them in a Discord call so Danny what else do any closing notes what else would like to say any um anything about Octoberfest anything about storybook anything about what maybe events you got coming up or con content uh yeah I mean I basically I guess I could just mention that uh you know I am I maintaining a few projects if you're you know interested in react native and storybook uh or even yeah it's it's a bit of a shame because my project is made a little bit more it's quite a bit more difficult to get started with if you're not familiar with like certain Concepts but you know if you are sort of interested in contributing to to a react native project um you know I have a I I maintain a project called uh for storybook react native so if I just go here to Starbucks correct native um always happy to receive contributions and there's also another one which is add-on Rec native web and that's the main thing and other than that um feel free to follow me on Twitter or and I generally just post about like storybook and Rec native and react so but my Twitter you can it should be in the link description in description below but I will but bring up on screen if you want as well or um I think if you share it in the chat it might YouTube might remove it but um okay well I can put the the app thing yeah do that exactly how do I can I type in this oh no you can't let me do it I'll I will type it leave that on the screen and I will uh I will do it for you um no thank you and um with with your with your projects on storybook um do you still use automated testing like Cyprus and so forth so what I tend to do right now like with my work projects for example I will use chromatic um and it's kind of like super it's very automatic it's full like for the application so like for example chromatic what I'll use it for is like visual testing and like uh smoke tests and kind of like accessibility tests you can do as well and then for stuff that you want to like sort of integrate the whole app together and do like an end-to-end flow then yeah Cyprus or or also there's another one was it playwright I think is playwright is really cool and that's what's being used by Starbucks uh test Runner which is the basically the what thing that facilitates you to do all of the like accessibility testing and stuff um through with storybook um and yeah like I think the easiest way uh to do stuff if you already have a stobic setup just check chromatic on there and it's all like automatic okay um if if you're not interested in adding another project in uh you can add some tests with the test Runner and then for the app itself something like Cyprus or playwright yeah it's like really useful awesome thank you so much uh so everyone that's uh Danny's GitHub and then I've just put on the screen and in the chat uh Danny's Twitter as well so drop him a follow um storybooks looks super interesting it's great to see that they're also thinking about accessibility and automated testing um yeah so much exciting stuff uh Danny thank you so much for that I see um Emma and others saying thank you as well which is good thanks for having me so um it's uh it was great to geek out thank you so much and hopefully yeah come and geek out with us in our Discord when you get time I know you're busy maintaining all those projects but um when is that when does that happen uh it's more more sporadic so but I will put it in in Discord it's got an event section now so I will log say tomorrow for example at two o'clock I'm going to you know share my screen and just work through stuff people and you know hop in and out um they can so you don't have to be there the whole time you can come in for 15 minutes if you've got 15 minutes and just you know hang out and so forth get some pull requests in so um yeah sounds good so yeah feel free join us in our Discord um and then uh yeah whenever you've got time to come geek out with us that'd be great awesome yeah thanks thanks for having me I had a great time uh it's like really really fun to be here great to show off some cool features and yeah thank you well sounds good we look forward to kicking out more I'm gonna bring up the holding screen uh Danny if you could hold on for two more minutes I can thank you personally after the live stream as well so everyone have a great weekend and uh keep going with Oktoberfest any questions come and chat to us in the early Hub Discord link is in description below and uh yeah we'll geek out with you soon thanks for joining everyone bye anyways [Music]
["anyways [Music] thank you [Music] hey everyone we have an exciting live stream today we are here with Daniel as our guest who's going to be teaching us storybook I keep hearing so much about it so I can't wait to see how it works I actually saw uh Daniel do a talk in Portugal on it and it was super interesting before we get into that Daniel please give yourself an intro and a shout out okay cool hey uh that's nice to be here thanks for having me", "on uh my name is Danny uh I'm also a British guy but I live in I live in Porto I work for a company called cover Flex We Do flexible benefits for like uh for employees and yeah basically I'm the the way I'm involved in storybook is that I maintain a couple of uh packages for react native and um yeah I uh just really came on storybook I like the community around it and hopefully I can show you a bit about it and um also storybook is a open source", "project so we know this is a hectober first thing and basically you know if you really like storybook you can also get involved on the storybook repository I love that so yeah you being a core maintainer you're encouraging people to contribute I think that's just awesome thank you so much Danny just to give a shout out to some people in the chat as well uh akshaya you were first welcome and we've got Emma Sashi and Carl and Kota prolu hopefully", "I pronounce the right awesome to have you all here we're actually going to be using the Eddie Hub link free project uh next JS Branch as an example here so we can see um if it's something we want to continue using or we can learn more about it and see if it's something that um uh it will be useful to us so uh yeah Danny I can say Daniel so Danny has has prepared a great live stream for us today so I'm super excited so what is Storybook I suppose", "yeah so storybook um is like a it's a toolkit for building UI and um essentially it gives you um some tools of course to to to build UI in a sort of isolated sandbox environment so you're able to like if you uh if you can see you're seeing my screen right now yeah that's correct yes cool so basically if you look at what's on the screen right now this is actually on the storybook website and uh nice enough to have a little GIF here with the it", "shows a little bit of an example here so essentially you can see um some like individual components on the screen and you can expand that into more complex components and or even you know go down to the lower level and each of these little things on the side here these are called what we call stories um or in fact this thing's here with the little book my bookmark icon and we'll see a bit more of that uh further in I'm sure and essentially you", "can do a couple of things here you can like change some of the props so if you're familiar with react you know you you know that when you have a component usually you have a couple of props that you want to change and you're able to do that without having to you know go into the code you can just uh they take a little toggle and I think they even have a little in this demo here they do a little thing where they change the time yeah he's doing it", "now and yeah basically you can use this in your development process um to basically simplify things and uh for example I know that in this project the link free project you're using a Docker image and you know uh that's pretty nice uh and it's it's like an easy way to get a environment set up but even simpler way to go is nothing right so if you just run storybook you don't even need the the database or anything running you're just using the", "components without the whole you know page around it and everything that sounds really interesting okay I'm super super excited now awesome and uh I think it'll really helping boarding onboarding people to the project they ought to see like we're not trying to go through the code see what components there are this will list what components are available to them if they want to create a new page and they want to use different components they can", "use existing components and just check what props are available um yeah exactly that's also like another one of those use cases for it which I didn't really cover just then but yeah you can do documentation as well in your storybook and some of the stuff that you know uh you saw in my talk from devtools Porto was about the you know the features like um automatic accessibility testing and like Integrations with figma and basically there's a load", "of different add-ons that extend the functionality of storybook to really make it that toolkit that you know you can use to help you build up components and we're you know um even things like mocking API endpoints so um I think we probably won't get too far into that today in terms of the API endpoints but that's something that you can consider as well and uh what the way you usually do it with storybook is you set up a mock service worker which", "is like a cool little package uh you know you can get from npm and it just lets you mock your API endpoints and if you have all your mocks set up you can even just plug that straight into storybook and it will work and and that's another thing you know like if you have that setup for storybook or you just have it set up for that so you can run like a mock version of your app they can share that config and uh it's in a little uh it doesn't", "require a lot more work to get that set up basically nice I think that sounds super interesting um people are already discussing in the chat about having yeah a sub domain where um uh we could um uh yeah have it deploy to and so forth so I think that's really good yeah awesome yeah and I think you can see a couple examples of that I I don't know if if you remember uh well I guess the chat won't know but there's a this if I just bring up this one", "example uh app which is uh basically uh this is just like a fake app for um showing off storybook and this is built by uh Geico Jan Braga and he's like one of the core Starbucks maintainers and so like you can imagine this is an example app that's based on like a finer restaurant sort of thing like a Okay Google Maps but just for restaurants or something you know and you can like order on here so what they what he's got as well is similar to", "that it's a It's like a if you go to I think slash storybook here I think it you probably would want it to be a subdomain right so that it doesn't really get that accidentally but then you can see here that app itself has its own Storybook in each one of these things you know like for example this is the footer that's from that app and then if you have that for your link free you know you could go oh wait how did they build that little link", "thing there and you go to the the storybook real quick and just see uh that uh component and you can even have a look in the docs and see like a it's not happening here but sometimes you could see like the source code oh here it is there's a show code if that'll work yeah you can kind of see something there but yeah there's some cool features like that cool oh I'm super excited so let's uh I know you've got some exciting plans for our link for", "you today so let's let's see I just realized the script was really small that's okay it looks good um I see the chat game quite quite excited as well we actually have a question from um from Carl uh is it using webpack or Veet for bundling yeah it's actually an interesting question uh so in the in the past uh it's been sort of a webpack only thing but um it doesn't stop you from using it with other bundlers like for example your app doesn't need", "to be using webpack for you to use storybook and storybook can be used like with things like Ruby on Rails but actually nowadays you can even run storybook with Veet so vit is that bundler that sort of came along with the uh View and the guys that built view right so and that you can use to get really fast uh hot module reloading in um or fast refresh I think they're calling it now with the uh with storybook and there's still some caveats to", "that where the currently storybook the the apps of around your components will be bundled with webpack but the preview where your components go you get to choose what bundles that so if you want to use V for that you can use it via like uh called the Builder V Builder okay makes sense Let's uh um that sounds good it causes us huge okay perfect yeah and actually you mentioned about slots I just saw his comment he mentioned about swell and yeah", "spell is like uh getting some nicer support nowadays I think as well I think we had some back and forth with some guys from the swell community and yeah the the support I think a lot of things are really going forward in a good direction for version seven as well so like version seven is coming up pretty soon and there's going to be some big improvements in a lot of areas including performances I think a lot of people worry about um especially", "when you get really big story books but you know there's already some tools you can use to get around it but in the newer version the the storybook UI is going to be pre-bundled uh but we might be getting a little bit ahead of ourselves yeah maybe it looks like Carl's saying so you've got another project called finder um and we use salt kit for that so they can see uh Carl's gonna test storybooks with uh with the finder project um which will", "probably be done in about 10 minutes because Carl is super fast he says I want to think about doing something before you know it it's already kind of done yeah it was just typing that out he was already in the code base yeah also so should we should we dive into the the project let's do it and I'm also thinking about you mentioned accessibility and I know Emma um is is a fan of accessibility so I can't wait to improve our link free project", "that's getting a lot of traction as well now awesome yeah so let's have a look first I guess we have this next.js Branch right this is yes but you've been building out like a is a new version as far as I understand right of the link free uh app and yeah so basically what we're gonna do is get on this branch and uh do the basic setup uh in the next JS project there's like one or two like extra steps that we will have to do but it's actually", "pretty nice because there's an add-on that sort of handles the complicated stuff so we don't actually have to do anything really complicated ourselves and um yeah me personally I never really worked much with next.js even Nexus with storybook but uh you know just looking around and figuring out it seems it's actually pretty simple and I didn't have any issues uh figuring out so I think uh nice if we go as we go through it here you'll see there's", "nothing super complicated here a lot of the steps it's just knowing okay these are the commands that we need to run um and actually have a well you know what let's I'll bring up the projects here so hopefully this is big enough everyone can see uh there's obviously not much to see right now but if I bring up the bring up some file in here that looks familiar yeah yeah oh let me just delete that we don't want anything sort of residual here yeah", "let's start from scratch so we can all learn yeah exactly I think that should be a clean State here and I'm on that so if anyone wanted to follow along I suppose they could as well and um so essentially what we want to do is uh run a little init command from storybook so uh storybook has this nice little SD CLI which of course sends this storybook and here if you run the init command um that will basically just set up everything uh the basics", "for you for storybook and at that point you can kind of already just run uh the storybook and it will have some sort of basic uh component setup from you know like example components they're not like from your app in that case and you have to add those yourself that would be really good that's in the next that's in the version AIDS yeah maybe we'll see how they how they feel about that awesome so yeah let's just run that and see hopefully it", "doesn't take too long um but it shouldn't do and you uh for anyone who's wondering what MPX is it's just like uh for instead of having to install this CLI you can just run commands from that package and it's a built-in feature of npm all right yeah thanks for clarifying that I make so many assumptions and so that's good that we shouldn't make assumptions so thank you yeah no worries so yeah now it's just going to go through and install a few", "things and if you look over here you can already see it's added back that folder that I deleted which you didn't no one saw that uh and so we've got a couple of files in here already well so whilst that's installing we could even just sort of Let It hide away for a bit and we'll have a look at like what files we have here if I just want to format that yeah so we have this main.js and that is funny enough the main config file and what goes in", "there is like some of the details about how your storybook is set up and you know getting started probably won't need to adjust much in here and we won't actually really touch that and if you saw in that command that I ran I had a little flag to say use webpack five I got you um that's I don't think that's super necessary but it's nice to use the newer versions and in Starbucks 7 they're actually deprecating webpack for but either way if you", "don't know what webpack is don't even worry about it [Music] yeah and if you wanted to use like the question earlier if you wanted to use you essentially just put like the hit okay you would have to install it as well so there'll be a package I think in this case we'll be able to see it uh I don't know maybe it's not finished it's not finished installing them yet but we can we could check that later maybe I think it actually just finished as I", "clicked away which is always uh always nice uh yeah so here you go I think yeah so we've got like build a webpack five and so you would in this case you would uh install V so this is not super important if you're if you're not sure about webpack and V but if you did want to know how to get the web Builder uh it's easier probably to just search on Google beat Builder storybook and you'll probably find everything you need to know but the basics is", "you just Swap this out and install the package so yeah other than that we have this stories um array here and this is basically like saying like where can I find the stories and the default setup when you run the command it's going to put them in a stories folder and it will look for anything any subfolders in there with and any files you know in the nested structure that have stories uh dot you know any one of these extensions so jsjsx TSX TS I", "think and this project is just JS so technically you probably only need those but um it doesn't hurt to happen more but yeah so basically this will tell storybook where the stories are and this is like a glob syntax so um I wouldn't worry too much about the specifics of it but it's kind of like a version of regex and cervical use that to find your files um so and then we have these add-ons here these are the ones that come by default when you", "when you set up a fresh project and Essentials here has a bunch of like really cool stuff like the the controls thing which is what lets you adjust props on the Fly um and a few other things but we uh we'll get into that I suppose when we actually run it um and just one more sort of thing here the preview we can set up a couple other things here and they're sort of like runtime config whereas these are more like uh bundle like uh build Time", "stuff so and and these preview things you can put here like jsx and things like that um so even here you can have like a decorators which are like wrappers for your component but um I think we can skip past that for now because it's not super important we won't need to use any of those things there's only one thing we'll have to add to this preview file and we'll get to that in a moment so like I said earlier as well we have this stories folder", "and they set up some example uh components and some example stories so if we just take a look here the there's for example like a page with a perhaps we look at a simpler on a button sure so that we have a button component um and this is like uh just a regular react component nothing about Storybook in here this is your if you had like a component in here you know this you could replace this with whatever other component and what you do in your", "story file which is what Storybook is looking for is you import that uh you know that component wherever it may be and then just tell storybook this story is about that component and this default uh export is what storybook's looking for for a lot of like metadata and things to just basically tell it what to do and a lot most of these are sort of optional things to be honest and the simplest story you just don't need any of this basically and it", "will give you uh a title and you know the args that's only if you want to use do the props on the fly so I'll if the simplest story you know if I just took like got rid of most of this it's just that basically and so the story itself is this here this export primary and that's essentially this template buying thing don't worry about it it's basically just exporting this template thing here which is a function that uh is returning that component", "and so essentially it's just a component itself um and so your story is like a component with a couple of extra things and then storybook picks it up and you know knows what to do with it so maybe a lot of this is probably not making a lot of sense right now but if we run the storybook yeah running I want to see what we've got we can yeah I just I was giving it time to finish running oh okay no worries I'm getting excited I'm like seeing this", "code I want to see how it looks no no you're doing the right thing so I get too excited no worries no worries no but but you're right to want to see it I think we should go straight to it if possible so I think there's a little thing that happens here because the s-link file isn't uh in the format it was expecting that we can just sort of um put it put this in there so that this is an optional step but storybook has a yes plugin so you could", "actually when you're writing your stories get suggestions about you know like oh you shouldn't do that because whatever I I don't know if I'll look in a specific example right now but I'm sure it will probably tell me something I'm doing is wrong at some point so now this has run all the way through it gives you a little output here saying yeah to run your storybook type um storybook and that's yarn is not required so don't worry about that we", "can just run uh with the npm syntax so yarn is like a just a a replacement or an alternative to npm so in case anyone's not familiar that's why it's saying to do yarn storybook it's the same as npm room storybook can I jump in with a question yeah you've got a question from uh Carl they say they found a blog post and Storybook of swelt but it uses version 7 Alpha would you recommend to use Alpha already or or not I think that really depends on", "you know what your you know your situation the project that you're working on like I I know a lot of like uh sort of more corporate setups you know where you you don't want to use anything that is sort of not uh stable and but like smaller startups or just personal projects it doesn't really matter as much you know what I mean and I think in the in the case of storybook which is more like a Dev tool it's not super important if it's not like a", "stable version you know what I mean like uh if if your storybook breaks it's annoying but it doesn't break for your users which at the end of the day is like the important thing you know so Carl I recommend if you found something good on the newer version which is it's gonna do you know when the newer version is coming out uh Danny it's roughly uh uh I'm not sure and I wouldn't want to give the wrong answer and that's the kind of uh thing to ask", "uh probably some of the the more of the web team uh no problem no I can't think of it like top of my head but sure no problem I think but it's gonna be released I think the the thing is the great thing is that these these releases are coming out so Carl I'd say go with the alpha um it doesn't break the app as also as Danny mentioned so um yeah so that's good yeah so and that's the thing like they are Super Active on developing that so you'll see", "like the releases like every every other day sometimes you know and they're just pushing out uh the new stuff and so they have full-time maintainers working on storybook uh storybook is sort of uh has a a company that funds a few people to be like core like maintainers full time so uh it's definitely it just keeps moving along so uh you don't have to worry about like an alpha being stuck in Alpha forever um that's good yeah are you able to make", "that a bit bigger it's a sort of full screen as well please okay it's okay is that is that good that's good yeah okay so now looking at this um one thing you can maybe do I don't know if this is a terrible idea but let me just see because you know this is sort of the thing that you might do when you're developing on Starbucks or what you can do if you have two monitors it's easier right yeah but I guess in all development you've kind of got that", "thing I do I put my thing over there on the left screen and then put my thing over here so just to sort of show like what happens what's what over here and over there you know so like I was saying um this is a story right so if I just put back whatever was here yeah okay I can sort of just explain what each thing is I don't know if I missed something all right here we go there we go yeah change the title that's why it was confused so okay okay", "so basically yeah I've got this button and this uh is a component just a regular rack button and I've told storybook that is the component and then I've said okay my story what I want my story to be is just you know render this component with the arguments that I passed through it and those arguments uh per story you can Define them and so in this case we've said okay for the primary one um it's uh let's set primary to true and so you can see", "when I clicked on this story um that's what's happened here it's set this to true so now if you see when it's false it's a white button and when it's true it's a it's a blue button and so this is like some of those like uh uh basics of storybook where you kind of just set up a state of your component and like a like a slice or how would I put a yeah I don't know like a I'm forgetting the word now but like I sort of a state or an example um", "someone help us out in the chat let us know um but kind of like uh okay we'll come back to that we'll come back we've kind of got a view of that component in that situation a possible state of this component uh like Frozen in time you can imagine but then at the same time we've got this control Z on which is one of the things that gets included by default and that lets you kind of take uh those args uh or which are you know in sober gland we", "refer to the what you might say are props in react we say uh args in storybook and so one of those arcs here is the primary Boolean Fields right and storybook has picked up the types of each argument and just said okay yeah this is a Boolean and this is a string and it'll do its best to figure out basically and that's why we've got a couple of things in here that we haven't even sort of listed yet but they're coming from you know uh when you", "pass this button saying that this is the component server goes and figures out like what are the arguments that I can pass to that and then it will sort of render those out to you in the controls panel and now I can sort of go in here and just say oh what does this button look like if I you know wrote like a whole paragraph in there maybe puts a new line caught you yeah and now you can kind of see like oh I'm just maybe I'm trying to like test", "all the edge cases for a component I can do that really quickly here because I don't need to go and like set up some mock data or something I just go ahead and type it in the browser nice by the way um Emma says it's a variants the different variants of the button yes exactly I would never have got that I don't know why that word would never have come to mind it's like so yeah and there's even some cool things here like we have this that's so", "cool background color changer thing uh Color Picker and um this comes up here well actually it Starbucks is able to guess even sometimes when it's a color prop so like if you give it a certain name so if we go back to the the preview that I sort of jumped past before if you look if the names it has this mattress thing which has some regex so if the name is background or color it will assume that the argument type is a color and then it'll give", "you the Picker but if it's not you can just force it to be that so that's why here I think because it's background color and not background they've just gone they've specified that is the con the color type so then so you if you even if it doesn't pick up the types that your props have you can sort of force it but yeah I mean and in the simplest version of storybook you can you can work completely without the controls even so if I just close", "that out and you just create these sort of Frozen uh States or these variants of your components uh set up by just like saying you know these are the args the default ones that I want to show when this renders so like for example this secondary one is like the same as the primary one except primary is false and so it's white you know in this case I need to reset the the the state of this so that it goes back to normal but yeah and then in oh", "whoops what have I done put that down close it yeah there we go sorry but yeah so in a simple storybook you could just set up these variants of your components and just flip through them and then something you can do is like you know maybe you have some automation to do some screenshots of each story and that's something that there are products that do that for storybook for example so you can have some automatic visual testing of like different", "states of your app or different states of your components um and yeah so like and you can see also here like some documentation um this is MDX built in you know that runs in storybook and MDX for those that don't know is like the markdown format if you're used to GitHub a lot of the stuff you type in to GitHub you can write uh in the markdown format and markdown that has an extension uh well there's an extension of markdown called MDX which is", "like jsx inside of markdown and there's a lot of like acronyms here but it's just basically saying you can put interactive stuff inside of documentation and you can put that into storybook so um and that's what this docs tab is all about as well so storybook could even generate some default uh documentation for you so I just full screen this I don't think there's there's nothing set up you know I haven't written any markdown for this button", "component but it's gone ahead and said okay we know this about this component and um they'll just render out in this nice little format and you can see like the different rarence here and you can see the source code I think I'm not sure I've not used this feature the the source code thing too much but I think there's a way to get to show a bit more information as well but yeah the the what's nice about it is like if you just wanted to come and", "see what the props are you just come here to the docks and see okay there's a this is a thing a primary is of type Boolean and the default is false I think that's great I think it just makes it like documentation takes time but if you have to have this documentation generated and then you're getting extra benefits people can actually interact with the component as well and I see you've got Pages there as well so it's just you're investing um in", "something that um and you're importing the component so it's more likely to be kept up to date as well I think like one thing that I think um it depends on your posture as well like if you're adding components to storybooks after the fact it can feel a bit like extra work but if you're working on your components from the beginning inside of storybook it just feels natural you know what I mean it's like I'm if I say like you were building link", "free from the beginning and you know you wanted to add a link component and you have a design for the component and you just come in here and you know you build up the component and then once you're done it looks at the way you want it to you can bring it over to your app or you know even have a story for that page um and then you know creating the story for it was part of the development process and it helped to you know debug things and Etc", "and and then you know it's not like that extra work you have to do it's just uh you know a tool that helps you whilst you're developing so so we have like tdd right so this would be like sdds a story driven development yeah actually sort of a thing that oh really I was joking yeah okay that's close I think so chromatic uh uh I guess it makes that bigger by the way yeah sorry so there's this article from gramatic which is component driven", "development and you know I think it's just that thing where like they had to do it of course there to do it because it's like um you know you've got yeah we've got test driven development and then yeah we do kind of have like a this idea of component driven development and I think it does make a lot of sense because when you work with react or you know a lot of uh Frameworks today that is kind of what you're doing naturally and when you put it", "into storybook it it's almost like it's not just components like um so I don't know if if you know about like the atom molecule what's that thing called with the I can't remember what they call it but the the storybook idea was sort of built up of this um this modular framework for building UI which is is sort of like where you have um you know your building bricks uh that are made up of like uh the smallest possible thing is an atom and then", "the next is a molecule I can't remember exactly you know what they what's that called but uh the idea is like imagine you know component development that's what you do right and react but if you take it to the next level and in storybook you'll take the component completely out of the app and just on its own in isolation you're truly just developing one component at a time and there's nothing sort of getting in the way or like I don't have to", "navigate through my app to get there right and I think that's like what they're getting at with like component driven development you know Ms saving the day again Atomic design everything so much this is right it's always good to have people in the chat right exactly I can't remember anything you were hit we hit I always say we're here to start the conversation and then other people can then jump in and um and add even more to the conversation", "uh before we get to the next bit um Carl asks does it add quite a few dependencies would you say it bloats the size of your project production blow you know it doesn't go to production these are all development dependencies and I don't think uh you know I don't think it's huge but it's no by no means like a little you know two kilobyte package and it also depends on how many add-ons you install like all the extra features that you're getting out", "of it so you know if if you know your project's already huge and maybe there's a trade-off there you might consider that but I think to me you know it brings enough value to say that it's worth it and none of that is going to my end users at the end of the day because you know I don't deploy the storybook as an app it's like I'm using it during development I can deploy it as a way of like showing off the designers or QA to get like their", "feedback on things but like I said before with the if it's down you know it's only for us internally and it's not uh so so like yeah there are quite a few dependencies and but like I think the trade-off is there like if it gives you value makes your development process faster let's say you know maybe add a minute to your install time but saves you you know a minute every day for the rest of you know your development time or something yeah so you", "have to kind of put it that way but yeah I would say it's not small but it's also not massive it's somewhere there in the middle sounds good I think it's you're trying to say that it adds value so it doesn't it's although it adds adds size but it adds value and it's not being deployed to production which I think is uh also important yeah and I think you just have to be a little bit careful that you don't allow those things it shouldn't by", "default nothing should go into your production build but just don't do something silly like import help the storybook package into your like component itself or something you know and then you should be fine and by default all of the documentation and all the like init commands and stuff will always install things as Dev dependencies in your project so you shouldn't get anything as like you know proper like production dependencies that sounds", "good right well okay so any more questions or do we should we no let's go let's go I'm super excited so yeah I mean I think we sort of come with a lot of the stuff that's in here but we could maybe also just fill it fiddle around with a few things here just to show you know like some of the other features that you get so out of the box right so we've got this little measurement um like inspect add-on that you can use without having to like open", "up the development tools and you can see you know this button is you know 44 by 14 and I think even if I bring up the accessibility thing uh maybe it's not got it here but anyway yeah you've got a couple of other tools you can even like zoom in here if you if you can't quite see the the difference in the colors or if there's like a one pixel thing you know that's always useful for like UI you can like refresh the state of the component so like", "if you're in a more complex component you might want to go back to like a normal State I don't know if there's one in here that no probably not but yeah um if I just disable that I think there's also yeah here we go so we've got things like you know adjusting the size if I just bring the zoom back to normal yeah we can adjust it to different viewports but we can also Define custom viewports and there's a viewports like default set of viewports", "that you can get from Starbucks as well you just have to add a small add-on that has like a list of like really good ones so like some like things like iPhone and stuff like that that you can use so all this stuff is like yeah I could open up my development tools and you know go through that but then if you look at sort of what happens in and you do that is you've got to sort of go through all of this uh where am I and you know in the storybook", "UI I feel like it's it's really set up in a really nice way so you can just quickly you know play around with it and you don't have to be developer to know how to use it which I think is really cool so like designers can can pick up on it as well yeah definitely bridges that Gap and you mentioned accessibility which I think again is is super awesome hey pretty much awesome to have you here um actually ask the question is it like WordPress no so", "this doesn't this is not building your app this is um helping the development of your app so it's so we can have a look at what components are available which we're going to get into now so you're joining it at a good time yeah so you know it's exactly what you say that you don't build in like the UI of storybook is just to let you visualize the stuff that you're building in codes it's not a no code tool you are coding but it will just add some", "like stuff in there for you for free and you know um I think if you feel like WordPress if you want something more like that it's storybook's not that kind of tool basically um but yeah let's get back into this I think I've basically shown there's a bunch of these little add-ons that come in here by for free and you can sort of see uh you know what each uh just little details about your component and sort of debug a few things like this one", "could be useful I don't know if you've ever done this when you're building an app but you're trying to figure out where that outlines of a component are and you just put down like borders around every single little component maybe you can kind of avoid that by using some of these features you know that sounds good yeah and just one more thing I suppose I could just show before we get into some setting up the actual project uh We've also got", "actions which are basically just um events you can track the events that are getting called when you click on things or type or whatever and so in this case I can see that when I clicked that I got an on click event and see some information about it and I think you can see more information for like more Complex events so that can be useful in some cases and the interactions thing probably will leave for another time but that's also a real", "feature that lets you do testing inside of storybook and in a visual way um so if you if we have time maybe at the end we can even do like a quick example and it might be pretty cool I think so let's actually set this up for link free and so right now what we've got of running these example components and these are like all the ones I have here these aren't just like um coming from nowhere these are so if I just deleted all of these then I won't", "have any components anymore and it will complain at me probably that's why just delete all of that because we don't really need that anymore uh so now we've got an empty stories folder if I go back here yeah it says your storybook is empty and it'll tell you that probably no story files or you know you didn't Define the right regex so um one more thing that I want to do before we do that so just so that we can get next yes working nicely um with", "the project if so if you remember we kind of uh run through this with this init command I update the this link and I'm basically just gonna um now install a little add-on for for next.js and what this does is just kind of there are some like Niche things in in next.js that are like really useful in when you're developing your app but in storybook uh it's a bit more like complicated so for example like next image component has like some special", "features for like caching and stuff and you know like server-side things and that's not really going to work in our setup where we're just sort of rendering react and and so this will just sort of solve that for us without having to um do any extra config so essentially all I'm going to do is run npm install and I'm going to save it to Dev my Dev dependencies so that it doesn't go into my production bundle um and yeah we just give that a second", "to install and whilst it's doing that we can do the other step we need to do once we've installed which is just add that to our add-ons list if we just come in here what all I've done is I've added this uh the name of that add-on and that will basically just tell storybook okay I need to call some code in in this package um and that's it essentially all of the config that you would have to do manually probably some webpack magic you'd have to do", "because it's sort of handled by this component and this is also something that's coming in version seven where next year's support would be like first class support so you won't probably even need this step uh in the future okay sounds good and is that in version seven on uh Alpha already do you know I'm not totally sure but no worries I can look at that later you should always give it a try and see what happens yeah the the main thing that", "you'll notice if you're not using it is basically the image component so if you're not using the next image component you might not even notice anything necessarily um but yeah we'll just see you'd have to try it out but for now I think it's safe to use this uh it's just like a default thing and it's it's a really it works really well like from from my experience let me see I think I had a never mind we'll go back to the hopefully it's gonna", "finish this installing soon so we can there we go nice just as I start complaining uh okay so one more thing that we're going to want to do since this project is a Tailwind uh it's set up with Tailwind we just want to make sure that those Styles get imported into store book or they get applied so one thing that I mentioned earlier is that the preview.js is sort of like a runtime uh config in a way or just like extras that you can add to the to", "your like previews um so that's why it's called preview because you know you have that component that renders in the middle that's what we call the preview in store book and if you want to apply anything to that part of storybook that's where you want to put you you're basically always going to the preview and in this case all we need to do is basically import these style files so if you look in This Global CSS you can see here there's some like", "Tailwind stuff and all this stuff we want in the storybook as well because otherwise we won't get those nice uh styles from the classes that you use in Tailwind right so if I just go and go to this preview.js and import that Styles files being the wrong thing I guess there we go so yeah I basically just importing those Styles and now type type Tailwind so it should work uh in um in the storybook so but we won't be able to say anything yet", "because we like we like I mentioned earlier we don't have any stories anymore um but so let's go ahead and fix that I guess um what I'm going to do in this case is add a folder to stories called components and in there I'll make a new file and before I do that I'll just mention what I'm what we're going to look at first uh there's a component here which is nice and simple which is why I'm sort of going with this one and it's just a little uh", "it's a little button component okay so I don't know let me see yeah so that's the we'll see it in a second when I bring it up but um we're just gonna make a story for this component and it does have some like requests in here but it's not super important if it fails so we shouldn't get any issues just rendering this without mocking things and this component takes in a couple of uh parameters so awesome props rather and so we can even you know", "make use of the args uh that storybook has and the controls there so we can you know adjust the username just by like typing in the UI so you know without further Ado I'll just add a file in the stories uh in the folder that I just created in stories I'm going to call it use the link dot stories oh JS and some of you might already be thinking well you know why don't you make it sort of alongside the component and that's actually something you", "can do so you know if I wanted to I can just go and update here so it'll also look in the components folder for stories um but in Nexus there's a this whole concept of like file based routing and so when we want to make a story for a page in our we can't just put it directly in the pages folder because it's going to assume that's a page of our app and so we'd end up having to make a separate folder for our stories anyway so in this case I've", "gone ahead and just said okay well let's have a a folder you know let's put all of our stories in the stories folder um but if you're not using xjs or you're not using file based routing routing you could put them alongside like I could just make a file in here called with the same name sorry I I do I do like it how it's in a separate folder though like some people put their automated testing in the same folder as the source code and I actually", "do like it like it's separate um maybe following a similar sort of structure um but have it have but sometimes it doesn't always make sense it depends on the project but having it separate I I prefer this way so this I think looks uh looks good I don't know what everyone else thinks in the chat let us know but um I do like the idea of having it in a separate folder yeah and think like it almost done to preference right and it shouldn't make any", "difference at all really other than just you know look and feel or you know Dev experience so if it feels better to you to have it alongside the component you can do that and you know you have that uh option in storybook so you're not limited to the stories folder um I'm just putting it there because in this case I think it makes a lot more sense for a nexjs project and so yeah we've got like stories file but we're missing a few things here so", "since this is like sort of the Crux of things let's let's do a bit of sort of live recording so I'm gonna go ahead and import that component so if we look here we're in stories components we've got to go back a couple of folders so I've gone ahead and uh you know that's why I've got these two dots and yeah I'm importing from components users user link and that's that component I was just mentioning which is a button right and so as I mentioned", "earlier what we want to do is basically tell a story with what our component is and we do that in the default export we don't need the title for now but what we can do is just say yeah the component is user link and you know that's why essentially the reason why we've been we've imported it there why is it I should okay so this is I said that the ear sling thing is going to come in handy somewhere and this is basically what it will do so you", "know you've got a story file you don't add any stories to it so it's not going to show anything and like yeah that's a good point actually I should add a story so that's what I'm going to do next so in in storybook the stories are just the exports and the default like the the named exports here so if I say like const uh yeah default sure copilot is definitely helpful yeah I know exactly and um so essentially this would be a named export and this", "is a default export right and the default export is like the information for storybook and the the default I mean the named export is the one um that is the actual story and it's kind of confusing because I've named this one default I'm gonna I'm gonna call it primary I don't know like uh I like to do basic I don't know uh but you can kind of call it whatever you want I'm terrible with naming this is where the community comes in and really helps", "but what we can always rename it later so basically is yeah and and you know that's the thing in programming naming is the most difficult problem of all so so yeah basically what we can do is this is like the basic um as we've named it basic as well it's the basic story it's just a it's a component that's taking arcs from storybook and passing them to uh the component itself which is the user link and then here what I can do is say yeah like", "it's profile is suggesting I can go ahead and say like oh here's the args for basic and what that means is like the default props that I want to pass to it so that when I click there you know that's the variant that we're you know trying to display and I have sort of set this up in advance and figure out what the user link is sort of looking for and so I know already what those are but and whilst you're developing is so you would probably uh", "sort of fig add them in as you go I think so here what we're looking for is a is a link and that has a URL so it can be Google and it has some clicks and uh also a name so we can say in this case I think what we're looking at are links you know on the link free page or links to content about you right so this would be like my YouTube channel or something right exactly and so this this doesn't make as much sense now if I just put youtube.com", "there you go and uh not sponsored by YouTube by the way but yeah so just username there's also another one that is expecting and then this uh displays that's public so if we just go and look here does exactly what I was talking about so link username and DisplayPort stats public um now this should be all we need to get this rendering in our storybook and what's going to happen is we'll see um this user link um will be the uh the thing that shows", "up in the sidebar and then it will have a little story called basic and instead of just talking about it let's actually run this and see our new story with userlink and we should see a little a button with my YouTube channel written there so we'll just give that a second to build looking forward to it so I just think it's so useful for people um working on the project or even new to the project to visually see um like kind of what the options", "are without having to dig into the into the code trying to think how people do it now and look at documentation that has been written might be our date so this is uh this is really good yeah and seeing from my own experience I had to sort of dig out what those props meant you know exactly what's a link you know what is it expecting inside the link you know because the other way that you would look for what these are is kind of go into if I", "remember correctly it's like a date not the data model link right yes yeah and so I had to sort of dig through that and if we've got these stories set up and you know whilst we're developing you have it in mind what you're expecting from that endpoint you can just set up the story and then the next person that comes along to try and you know add to your uh component knows okay well it's got these compo these things already I just need to you", "know maybe you want to add like uh I don't know whatever here something like a description I don't know and then you just all you have to do is sort of figure out whether the name's coming from and and then you can add a description and but you don't need to figure out much else uh but yeah I think it's exactly like that you know the you can also like I said see the components that already exist and maybe reuse some stuff uh and instead of like", "creating two and some later on someone goes why haven't made that component yes which does happen yeah exactly so yeah and you can do a lot of custom stuff as well like you could have like one page at the front with like a bunch of your components just showing off on the on the documentation page so here we go enough stalling here we go we have a link being rendered so that's you can as you can see that's what I wrote in the story itself my", "YouTube channel if I want I can just come in here and say you know uh twitch you know or something and you already you can see an update and in this case here we have like a sort of more nested structure for this prop and that's why we've got this sort of Json editor coming up here and it's pretty nice actually that it's kind of picked that up and said oh here you go you can sort of edit that in a nice way but um I guess these ones aren't", "actually visualized in the component they're more useful like when you click it it's gonna like try and make a request right so yeah but yeah I mean that is essentially like you're set up now you've got that story in there and now everything else that you want to do you can do from there and let's you know if you were developing this and let's say you know you wanted to I don't know maybe change the color or something now you've you know you've", "you're in a situation where you could easily you can see that component in isolation as well I think yeah exactly you could easily see okay well um I don't really know any of these things but let's get rid of one of these or something and I can say how does it look if I get rid of that got you no and I don't have to you know navigate through an app or run the docker uh you know I'm just here already like a new user I wouldn't have to go through", "the steps that I went through today you know like I've installed storybook and everything they just come in you know run npm install and then run npm run storybook and then they can you know already get going and updating some components no it makes complete sense I'm really really liking this um especially when you have different versions variants I should say of of the component I think is awesome yeah exactly and I I think it's actually um I", "think we discussed this a little bit before but I think it's a really cool way of getting people uh you know getting new contributions as well to a project so doing your first contribution so like your first contribution could just be adding a a new story for one of the components exactly and you can get your hacktoberfest going basically I'm liking that oh yeah this could be an idea for some uh some tweets later in the week so perfect nice", "right so um what I was thinking next is we could do a sort of more complicated um component and in this case an actual like a full page so um it's unfortunate I don't have the maybe if I can just go to link free like I'll go to your link for you maybe and show you like what link3 actually looks like because I don't know if I'm I suppose people will know right most most should uh the the downside is the the version of xjs branch is not deployed", "um so uh but it's okay you don't have to show it but they'll see it within um within here right yeah I'll just do it real quick just so that we know like what we're expecting to see a rough idea is not exactly like this everyone because I haven't done icons yet there's no colors yet so um the new version going from react to next JS does have a is not as feature-rich at the moment but it does have other features like statistics and clicks and", "stuff like that but that would be perfect for some some contributions right exactly yeah so if you see stuff that's missing here you can go in and add it right so but yeah and I think like so what we're going to kind of try and do is get you know like a base this basic page with the features that are in the next JS version showing in storybook and in a way that's just kind of like a mock version of the app running locally in a sense this app is", "a bit you know it's kind of like a a one-page app in some ways right because you've just got that I don't know that well I don't know if there's some hidden stuff in there but um we can kind of show the whole app in storybook even just by putting in the I think what's the page called uh let me see it's the username page I guess um so if I go back to my vs code um and we'll maybe I'll even zoom in a bit more just so you can see what I'm looking", "at in the sidebar here so if we look through this project we've got the pages folder so and this is that thing we were talking about earlier is the file based routing and the username file is um this is a page depending on which user is and it's going to show their their link free page right yeah this is the one that we're gonna try and make a story for okay cool and this one is kind of interesting actually because it does do some uh server", "request stuff but it's doing it on the get server side props function so and for people who don't know about Nexus that's just like do this on the server but our actual component is here and then it will get the props from the server side from this function right and I'm not super familiar with Nexus but I think that's how it works so this data will be coming in in the real app will be coming from get server size props right yes so basically you", "don't need to do the skeleton loading like you're doing react because the component won't be displayed until it's already got the data um so we actually had someone add to the branch skirt and loading and then put a sleep of like 500 milliseconds to try and so you could see otherwise you would never see a skirts unloading and I was like no we don't we don't need it we want to get away from that whole skeleton loading stuff that's okay and so but", "the reason I bring it up is because that's actually kind of useful for us right now because every thing every bit of information is actually in this data object so what that means is if we want to render that in storybook all we have to do is take this user uh component okay and pass it the the props that would be would be you know present in the database essentially correct would be the response from get server side props so I don't even need", "to Mark anything in this case I just you know put some args there or the one ARG which would be data right um and that's pretty handy and if we go through here there's you can see already there's a couple of things here like name Avatar um you know the links which you know and this is interesting because what we've just done is looked at the user link right and now what we're going to do is we're going to sort of compose a bunch of components", "together in one component and we kind of you're seeing the building box that you can build up like the atomic design that Emma so nicely reminded us of and and this is like this is that uh you know the big the the final product in a sense and if we had each of the components used here in their own component you could see like the whole all the building blocks coming together in a sense and and in this case what we're going to do is see the whole", "thing as a whole and to do that well what we're going to do is add another file and another folder even to our stories folder so I'm gonna go back one but you know what it's fine let that be that big let's go okay let's make pages so what I'm trying to do here is kind of follow the structure of the the app so in the stories folder if I wanted to find something I would and let's say Okay I want to find this component I just follow the same", "structure in here so perhaps I should have made a user folder now I think about it but you get the idea yeah we can we could even do that real quick I'll just put user and then put that in there it might even update the inputs for me it did it did okay nice so that should continue working so that was the idea anyways like you just sort of copy the structure that you have throughout the project and you'll be able to easily find stuff when you're", "looking for it and you know that's the there's a trade-off here like when you're if you put it like alongside your components it's really easy to find them when you're working on it yeah you have to do a little bit more work but anyway we'll get back to the storybook stuff and in this case we've got pages and then just straight up just straight the the folder there and so we can we can give it the same name as it has oh you can give it the exact", "same name interesting yeah I mean in in these cases like uh you know your founding can basically have anything in right and sorry book's not stopping you from doing anything as long as you give it you know in your main.js just configure it to say like you know this is what I'm looking for sure we've got to hear this star dot uh stories and so it could be literally anything there as long as it's got stories on the end and uh one of these", "extensions so yeah we can now go ahead and just for example import that user component which I think is the default export me you know I think too many things open now so I'll just close everything else and bring up username so if we look here that is the default export so I can just straight up import it like this and be nice if here we go yeah pages and then username I think you need to go up oh yeah you're right yeah yeah three things yeah", "you're right I even have it next to me and I'm still doing it wrong no worries it happens so what is it complaining I think it's because I haven't used it yeah there you go so yeah that's the right path now we just do the same as we did in the other one I'm gonna give it a default export with some information I'm gonna put the user in there which is the component and just close that off and that that again is just the the basic stuff we can we", "can give it a title and even just say this is you know and say that it's the the user component but it does that by default but what you can do here is kind of like um you could even give it some sort of more complicated structure and say like uh but what they've done recently is made it so that it will automatically check your file structure and then put it under like Pages user so I'm not sure if we'll see that here but I think it should do", "that once we have more than one component we'll be able to see that a bit better and so now like it was complaining there you know we have to have at least one story and so what we're going to do is create again our basic doesn't it can be called anything but you know as I normally like to create basic just for whatever reason and so we're gonna do the same thing again which is just pass the args to that component and if you don't do that it", "will also work it just uh won't work with the controls and actually with the newer version of storybook um you don't need any that at all you can just pass an empty object and it already work um this is uh what is coming as part of csf3 which is this this syntax we're writing here is what they called CSF and we can actually use it already it's kind of in a preview state it already works in the current version of storybook so you know I think in", "this case let's do this one with the with that syntax maybe or would that be to would that be like mixing up too many things yeah yeah keep it simple keep it simple it's I've only got a small brain I can only cope with so much okay so we're going with like the standard uh like this F2 way of doing things and but you'll see in the future as 7.0 comes out for anyone who's interested anyway then there will be changes um and it will be going towards", "this new format with csf3 where it's actually really simplifies the uh the editing experience of of stories where you half of these things you don't even need to write anymore and like you'll see that this is a kind of like a repeated pattern that we do a lot of the time with the args passed to the component and in the newer version you don't need this at all it just does that by default assumes that's what you want to do and then if you want to", "do something different you can tell it but anyway yeah we'll go with the basic stuff and so here you know we passed the args and in this case there's a lot more that it's expecting so I'm just going to copy from an example I have um let me just get that I think this should be it so in this case we have a few more details we want to pass yeah we've got the links and we've got the the data and everything and but yeah this is essentially everything", "that we had to do we just set up that named export and then we set the args for it and these are all the default values like we were like I mentioned before and because it's in our stories folder and it's got the stories extension and those are the things that we Define in our main.js like where we're looking for stories it should now appear if we're lucky it should now appear in here oh something's happening hopefully I haven't done something", "wrong if nothing goes wrong it's not a live stream so it's it's good never be questioning if it's live or not so one thing you can sometimes do just to make sure is just sort of shouldn't have to do this but sometimes it will catch errors if you just sort of restart the the server the Starbucks server um but generally you know it has hot module refreshing and everything so um if we just give it a second here we might find that there's something", "sort of error here that where I've put the import wrong or something yeah it might need one more level up I think I think you might need that but I'm not sure I'm just gonna answer um some questions I'm just going to say while you're debugging it yeah so we're possibly going to add this to link free um so therefore there could be some uh maybe the next JS Branch the version two of Link three um so that would be good to be thinking of adding", "storybook to that this is like an example that Danny's taking us through with link free still but if we go ahead and do it then we're going to need to add all the existing components into the storybook so yeah any help on that appreciated and it will count towards Octoberfest as well okay awesome yeah so I figured out what the problem is if you see here I've put a little comment here saying replace me so that's basically why I forgot to do even", "though I've reminded myself in the guide um but basically in this uh uh page there's a picture of the user and so it's expecting to receive an image there so what I'm going to do is essentially just add a an image there or I can give you or I can give you a URL to an image as well I I have a I think I have one sort of set up here somewhere okay just it shouldn't take so just take a second here if I just find I'll just save it again but yeah I uh", "I knew this is gonna happen that's why I wrote that there no problem uh so Vijay says storybook really takes a commit author to the next level yes so um hopefully it uh is gonna help our project but we're interested to know what people's thoughts are we'll continue the conversation in Discord I'll do some screen shares in Discord next week as well um oh so this is the little placeholder image I found online okay uh this should probably be", "something you know uh be something else because I don't know if this is like a public domain or not but just for this demonstration sure yeah or use that and this is actually a PNG in this case so it should now stop complaining um so if I go back refresh oh it's because the Rand failed the Run failed here so we just run that again we sort of you know pray to the coding Gods we should get you should get this working um yeah this question about", "Oktoberfest but I do so much content Twitter space live streams on it and if we get time we'll chat about it in a moment but not just yet I want to focus on link free project and um and storybook and this is how you can get involved in uh hack type of Fest as well ah we have a Pages section yes exactly so that's the thing I was saying Starbucks now is sort of checking your file structure and it will organize the stuff for you but if you want to", "create your own organization you can also do that and if you just add slashes in the in the title field okay on that metadata it it assumes that is the structure that will take that as the structure and create folders out of that essentially so in this case we've got that uh user thing let me just awesome there we go so this is like you know the most basic version of the link free uh sort of that single link free page with this data and now if", "you wanted to see you know uh the specific Fields here and like just put like I don't know exclamation mark here you can see already what it's going to look like with different links in there basically no that's great and the um and if people I know someone's added like dark and light mode to not to the the next JS version to the react version but um they'll be able to yeah see that so yeah I think there's just you know different as people add", "features to components they can see they can kind of see it so I think that's awesome yeah and if you if you kind of sort of if you go ahead with storybook and you want to you know make use of uh those themes and like preview the themes in your storybook there's I think there's something called like Dean's picker or something like that or you can set up a pretty simple way of just swapping themes if you that mail drop example that I showed", "earlier has one that was set up by Yan um if I just zoom in on this I think there's something in here where you can oh good yeah so in here he's added a this thing here and this is a theme picker basically and he's even added a little thing where you can put them side by side that's very cool so yeah like little things like that if you have a dark mode or all that kind of stuff you can try and you can visualize that and imagine if you're", "developing um you know maybe let's say an app you would go and like oh I have to change the dark mode each time I want to like change uh you know the styling or I want to add a new component that works for both modes if you have this feature for example you can see both at the same time pretty easily so it's a nice way of like facilitating that that looks really good I like it um I really think I'm looking forward to chatting with the Eddie Hub", "community in Discord and seeing um I definitely think we should uh do especially the next JS branch on link free um do some more on this um because I think it just adds yeah so much benefit to onboard people I have to look at the mocking and so forth because when you click these buttons it's going to create an API request um to to do the stats I'll have to have a look at how to do that but it sounds interesting yeah I mean if you like I've done", "that kind of stuff with mocking before so if you need any sort of guidance on that I can help you out but essentially um I think I have a couple of links as well that I added to the that little notes uh thing that I sent you earlier but essentially um you want to use I'll bring it up on screen here there's this add-on called MSW uh storybook add-on okay and in here you can basically forget to an example here you can just say like um if I zoom in", "a bit more so let's say we had like a a get request to user and you can just sort of say okay when this request goes off just you know intercept that and send this response instead and so yeah this is pretty nice all you have to do really is set up uh install like MSW and install um the the add-on for Starbucks that's uh that looks really really cool um yeah it looks great I think we could definitely do that is that in the documentation on on", "storybook yes so they have a bit a little section um I'll just bring it up as well now that we're yeah they have a [Music] um a page on the on the official documentation here if you come to this if I zoom in again um we come through this um storybook documentation page so it's on the storybook.js docs you can also if you can go to it pretty easily from the from here you just come to the Box okay uh I guess one of these things guys I think it's", "guides yeah so in here there's a little section about writing stories and they have a section on building pages and screens and so like in general like for just basic components you won't gen normally won't need that kind of mocking stuff but when it comes to like building Pages there's a couple more things that you might want to consider right and and in this case they sort of list out some of the stuff that you might want to think about so um", "if we go down here a bit you can see like mocking connected components and mocking API Services Etc and then here they basically suggest that MSW add-on and then they've got some of those examples here as well um but yeah like I think the documentation um you know storybook is really in depth you just have to sort of pick it out of the uh in those pages but anything you need I'm I'm willing to help out and thank you so much maybe I can make a", "piano for you yeah I was gonna say maybe with the stuff you've done already maybe just uh submit that as a PR now um and uh that can count towards your Oktoberfest nice yeah I actually recently did sign up so far it's just been my own PRS but this would be good to get like a a real one kind of uh in the bag uh so let's see um to do that let's go ahead and make a new Branch I guess yeah and then you probably need to add it as a remote as well", "yeah yeah but I have one already I think oh okay well normally what I do actually is I use the GitHub CLI and I'll just do like create PR or something oh I haven't used the GitHub CLI so um yeah uh that's pretty cool yeah I'll I'll work in a demo right here okay perfect so um just just to check through the things that I've done here I don't need to check that so I think that's good to go so make sure we don't put any sort of rubbish in in the", "project I don't know it's kind of working with limited space yeah so yeah so I'll just go through and check real quick I think it seems okay got Starbucks stuff it's great advice to check to check your changes before committing that's good you never want to push a console log like I think probably sixty percent of my PR's go to my colleague and get a comment immediately you left a console lock yeah or when people do uh as yours is just joking it", "was just uh you know git add dot yeah never use dot um you know probably for the sake of like brevity I should probably just do that right now but in general okay I can get rid of this yarn lock yeah yeah um this seems I think everything's okay so what we'll do is just to get but and I'll do a New Branch I'll call it uh let's see I've got a feature storybook or something yeah and then do the do the merge do the pull request into next JS branch", "yeah yeah I will do okay so I'll just make a new branch and I'll do mats with these changes so let's do add so set up storybook and I think that should be all we need to do then like you said it'll have to put it on my Fork of the project so what I normally do for that is just creates uh I think that should be enough yeah so I don't want to push it to my branch on my Fork rather uh it looks good this is pretty cool yeah I need to just check that", "I'm I have some like weird issue with my gut where it doesn't always authenticate so I'll just authenticate off from the side okay and with the template you can just choose a blank one it's fine yeah okay blank one yeah it's fine okay we'll create an issue and I'll update the description so don't don't worry too much about it yeah so I'll just uh what I like to do usually is continue in the browser and then what it does is it opens it up um and", "now you can see I should probably not open another browser but here we go uh here we go so it's saying right now oh my God it's so zoomed in I can't really zoom out slightly okay so I just need to change this now to next.js oh yes yeah and it should now yeah now it can merge oh my God why are you doing this Okay cool so I ended up using the template anyway but you can just leave it you can just leave it it doesn't matter it's not fixing an issue", "like [Music] uh and and storybook to the next stress branch that doesn't really make any sense to be honest but it's a different branch you could list the the two components that you did yeah you could say um uh link storybook and or link was it user link user link that's the one and uh and you did the page as well oh uh user profile page yeah cool so I think that's kind of all we need if we go down here we can even see I haven't made any stupid", "mistakes and the only thing I'm not sure about is probably you want to put like a different image there oh yeah the picture will probably want to yeah delete that so but that's fine we can we can we can do that later cool so now we just this is almost like a demonstration how to make a progress as well but don't probably don't delete everything in the template if you're doing it yourself yes don't do that this is just for the live stream and and", "to the people out there I don't know if you ever come across this but oh I love it when people allow edits by maintainers because if you don't it small little changes are really like annoying to get through yeah and if someone is not available you can't make changes to their PR to just like get it over the line but enable this that's what I want to say I run into that a lot but yeah so we'll create this pull request and there we go let's see one", "thing you could do um is if you wanted to you could set up chromatic and then you'd have a storybook Builds on every pull request depending on how you set it up and then you can open that storybook live uh from here you can add a link to the storybook with those changes from the pr and then you can check someone's changes to the component's life and and something I was going to mention it because they actually have a open source uh like pricing", "thing where you can and you can get it for free if you're on a source project okay so it's worth looking into so yeah look free for open source oh I shouldn't if I click that then it brings up a chat but so if if that's something you want to go down that road I highly recommend applying for the open source um license or whatever it's called and yeah this is really cool because you can do all those free you can get the visual testing for free you", "can get the the hosting for free and like you can you know that act you get an action for GitHub that will build the storybook for each PR and that will sort of just take your do all your CI stuff for you and you skip all the the setup okay and so we will have a look into I suppose later on when we get a yeah you know a bit further down the line um I'm sure we could do something with GitHub actions like a basic version now just to yeah for sure", "and there there's one thing called uh test Runner will run through stories and like do quick checks on them so that's also an option but yeah if you you could you could make a build of each storybook and just see that it doesn't fail probably be the simplest thing yeah which is which is which is important I think it's uh yeah good to check that so um this looks good so off the live stream or probably tomorrow I will um get this merged delete the", "image to have a bit of a play and uh we'll work on this in in Discord I mean this is uh pretty pretty awesome I really like it while we're just doing a closing note because you switch back to the tab that actually has the the storybook because people are joining now Carl's back hey Tom hey shivangi so yeah leave that so this is what the version of Link free version 2 looks like at the moment it's got the number two in the links that's how many", "times it's been clicked so yes it doesn't look as nice as version one yet but I was going more for functionality and we're moving to Tailwind so actually having to to build this ourselves but using storybook will be great for us to reuse components um and know what availability there is within the project so um big shout out to Danny for taking time you know hour and a half to the minutes of perfect timing um taking us through uh storybook I I", "really really like it I really can't wait to start using it and then look forward to geeking out everyone next week um on this with them in a Discord call so Danny what else do any closing notes what else would like to say any um anything about Octoberfest anything about storybook anything about what maybe events you got coming up or con content uh yeah I mean I basically I guess I could just mention that uh you know I am I maintaining a few", "projects if you're you know interested in react native and storybook uh or even yeah it's it's a bit of a shame because my project is made a little bit more it's quite a bit more difficult to get started with if you're not familiar with like certain Concepts but you know if you are sort of interested in contributing to to a react native project um you know I have a I I maintain a project called uh for storybook react native so if I just go here", "to Starbucks correct native um always happy to receive contributions and there's also another one which is add-on Rec native web and that's the main thing and other than that um feel free to follow me on Twitter or and I generally just post about like storybook and Rec native and react so but my Twitter you can it should be in the link description in description below but I will but bring up on screen if you want as well or um I think if you", "share it in the chat it might YouTube might remove it but um okay well I can put the the app thing yeah do that exactly how do I can I type in this oh no you can't let me do it I'll I will type it leave that on the screen and I will uh I will do it for you um no thank you and um with with your with your projects on storybook um do you still use automated testing like Cyprus and so forth so what I tend to do right now like with my work projects", "for example I will use chromatic um and it's kind of like super it's very automatic it's full like for the application so like for example chromatic what I'll use it for is like visual testing and like uh smoke tests and kind of like accessibility tests you can do as well and then for stuff that you want to like sort of integrate the whole app together and do like an end-to-end flow then yeah Cyprus or or also there's another one was it", "playwright I think is playwright is really cool and that's what's being used by Starbucks uh test Runner which is the basically the what thing that facilitates you to do all of the like accessibility testing and stuff um through with storybook um and yeah like I think the easiest way uh to do stuff if you already have a stobic setup just check chromatic on there and it's all like automatic okay um if if you're not interested in adding another", "project in uh you can add some tests with the test Runner and then for the app itself something like Cyprus or playwright yeah it's like really useful awesome thank you so much uh so everyone that's uh Danny's GitHub and then I've just put on the screen and in the chat uh Danny's Twitter as well so drop him a follow um storybooks looks super interesting it's great to see that they're also thinking about accessibility and automated testing um", "yeah so much exciting stuff uh Danny thank you so much for that I see um Emma and others saying thank you as well which is good thanks for having me so um it's uh it was great to geek out thank you so much and hopefully yeah come and geek out with us in our Discord when you get time I know you're busy maintaining all those projects but um when is that when does that happen uh it's more more sporadic so but I will put it in in Discord it's got an", "event section now so I will log say tomorrow for example at two o'clock I'm going to you know share my screen and just work through stuff people and you know hop in and out um they can so you don't have to be there the whole time you can come in for 15 minutes if you've got 15 minutes and just you know hang out and so forth get some pull requests in so um yeah sounds good so yeah feel free join us in our Discord um and then uh yeah whenever", "you've got time to come geek out with us that'd be great awesome yeah thanks thanks for having me I had a great time uh it's like really really fun to be here great to show off some cool features and yeah thank you well sounds good we look forward to kicking out more I'm gonna bring up the holding screen uh Danny if you could hold on for two more minutes I can thank you personally after the live stream as well so everyone have a great weekend", "and uh keep going with Oktoberfest any questions come and chat to us in the early Hub Discord link is in description below and uh yeah we'll geek out with you soon thanks for joining everyone bye anyways [Music]"]"""


In [10]:
type(documents)

str

In [11]:
r_splitter = RecursiveCharacterTextSplitter(
            chunk_size=450,
            chunk_overlap=0, 
            separators=["\n\n", "\n", " ", ""]
)
docs = r_splitter.split_text(documents)
print(docs) 
print(len(docs))

["anyways [Music] thank you [Music] hey everyone we have an exciting live stream today we are here with Daniel as our guest who's going to be teaching us storybook I keep hearing so much about it so I can't wait to see how it works I actually saw uh Daniel do a talk in Portugal on it and it was super interesting before we get into that Daniel please give yourself an intro and a shout out okay cool hey uh that's nice to be here thanks for having me", "on uh my name is Danny uh I'm also a British guy but I live in I live in Porto I work for a company called cover Flex We Do flexible benefits for like uh for employees and yeah basically I'm the the way I'm involved in storybook is that I maintain a couple of uh packages for react native and um yeah I uh just really came on storybook I like the community around it and hopefully I can show you a bit about it and um also storybook is a open source", "project so we know this is a hectober first thing and basically you know if you really like 

In [28]:
import chromadb
from chromadb.utils import embedding_functions
CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs"
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [29]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

In [30]:
collection = client.create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_func,
        metadata={"hnsw:space": "cosine"},
)

In [34]:
type(docs)

list

In [36]:
collection.add(
    documents=docs,
    ids=[f"id{i}" for i in range(len(docs))]
)

In [38]:
query_results = collection.query(
    query_texts=["what is Tailwind css"],
    n_results=3,
)
query_results


{'ids': [['id279', 'id93', 'id281']],
 'distances': [[0.41365355253219604, 0.422451376914978, 0.4819260835647583]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['Tailwind uh it\'s set up with Tailwind we just want to make sure that those Styles get imported into store book or they get applied so one thing that I mentioned earlier is that the preview.js is sort of like a runtime uh config in a way or just like extras that you can add to the to", "your like previews um so that\'s why it\'s called preview because you know you have that component that renders in the middle that\'s what we call the preview in',
   "Tailwind stuff and all this stuff we want in the storybook as well because otherwise we won't get those nice uh styles from the classes that you use in Tailwind right so if I just go and go to this preview.js and import that Styles files being the wrong thing I guess there we go so yeah I basically just importing those Styles and now type type Tailwind

In [40]:
query_results['documents']

[['Tailwind uh it\'s set up with Tailwind we just want to make sure that those Styles get imported into store book or they get applied so one thing that I mentioned earlier is that the preview.js is sort of like a runtime uh config in a way or just like extras that you can add to the to", "your like previews um so that\'s why it\'s called preview because you know you have that component that renders in the middle that\'s what we call the preview in',
  "Tailwind stuff and all this stuff we want in the storybook as well because otherwise we won't get those nice uh styles from the classes that you use in Tailwind right so if I just go and go to this preview.js and import that Styles files being the wrong thing I guess there we go so yeah I basically just importing those Styles and now type type Tailwind so it should work uh in um in the storybook so but we won't be able to say anything yet",
  'Tailwind right so if I just go and go to this preview.js and import that Styles files being th